# Ajuste de Intensidad
Notas Importantes:

- OpenCV and Matplotlib expect different orders of the RGB channels
- Matplotlib Orders: RED GREEN BLUE
- OpenCV Orders: BLUE GREEN RED

## Librerías

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import cv2
from PIL import Image
from google.colab import files
import requests
from itertools import combinations

## Leer Archivos

In [ ]:
uploaded = files.upload()
image_path = list(uploaded.keys())[0]
imagenes = []
for filename in uploaded.keys():
    img = cv2.imread(filename)
    imagenes.append(img)
type(imagenes[0])

### Escala de Grises

In [ ]:
gray_imgs = []
fig, axes = plt.subplots(2, 2, figsize=(15, 15))
for i, ax in enumerate(axes.flat):
  if i < len(imagenes):
    gray_img = cv2.cvtColor(imagenes[i], cv2.COLOR_BGR2GRAY)
    gray_imgs.append(gray_img)
    print(gray_img[600,500])
    ax.imshow(gray_img, cmap='gray')
    ax.set_title(f"Imagen {i+1}")
    ax.axis('off')
plt.show()

## Procesado del Histograma

In [ ]:
rgb_imgs = []
gray_imgs = []
for i in range(len(imagenes)):
  rgb_img = cv2.cvtColor(imagenes[i], cv2.COLOR_BGR2RGB)
  rgb_imgs.append(rgb_img)
  plt.figure(figsize=(12, 7))
  plt.subplot(2, 2, 1)
  plt.title("Imagen Original")
  plt.imshow(rgb_img)
  plt.axis('off');
  # Pasamos a Escala de Grises (Un solo valor por píxel donde 0 es negro y 255 blanco)
  gray_img = cv2.cvtColor(imagenes[i], cv2.COLOR_BGR2GRAY)
  gray_imgs.append(gray_img)
  plt.subplot(2, 2, 2)
  plt.title("Imagen en Escala de Grises")
  plt.imshow(gray_img, cmap='gray')
  plt.axis('off');
  plt.subplot(2, 2, 3)
  color = ('r','g','b')
  for i, c in enumerate(color):
      canal = rgb_img[:,:,i].flatten()

      plt.hist(canal, bins=256, color=c, alpha=0.8)
      plt.title("Histograma RGB")
      plt.xlabel("Valor RGB")
      plt.ylabel("Frecuencia");
  # Aplanamos matriz
  vector = gray_img.flatten()
  # Histograma
  plt.subplot(2, 2, 4)
  plt.hist(vector, bins=256)
  plt.title("Histograma Escala de Grises")
  plt.xlabel("Intensidad")
  plt.ylabel("Frecuencia");

### Ecualización del Histograma

In [ ]:
for image in imagenes:
    # Separar los canales R, G, B
    canales = cv2.split(image)

    # Aplicar ecualización de histograma a cada canal
    canales_ecualizados = [cv2.equalizeHist(canal) for canal in canales]

    # Reconstruir la imagen RGB con los canales ecualizados
    imagen_ecualizada = cv2.merge(canales_ecualizados)

    # Normalización posterior para asegurar valores en el rango [0, 255]
    imagen_ecualizada = np.clip(imagen_ecualizada, 0, 255)
    imagen_ecualizada = imagen_ecualizada.astype(np.uint8)

    # Configurar la figura y tamaño
    plt.figure(figsize=(18, 12))

    # Imagen original
    plt.subplot(3, 3, 1)
    plt.title("Original")
    plt.imshow(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
    plt.axis('off')  # Eliminar ejes

    # Imagen ecualizada
    plt.subplot(3, 3, 2)
    plt.title("Ecualizada")
    plt.imshow(cv2.cvtColor(imagen_ecualizada, cv2.COLOR_BGR2RGB))
    plt.axis('off')  # Eliminar ejes

    # Histogramas de la imagen original
    colores = ['r', 'g', 'b']
    for i, canal in enumerate(canales):
        plt.subplot(3, 3, 4 + i)
        plt.hist(canal.flatten(), bins=256, color=colores[i], alpha=0.7)
        plt.title(f"Histograma Original ({colores[i].upper()})")
        plt.xlabel("Intensidad")
        plt.ylabel("Frecuencia")

    # Histogramas de la imagen ecualizada
    for i, canal_ecualizado in enumerate(canales_ecualizados):
        plt.subplot(3, 3, 7 + i)
        plt.hist(canal_ecualizado.flatten(), bins=256, color=colores[i], alpha=0.7)
        plt.title(f"Histograma Ecualizado ({colores[i].upper()})")
        plt.xlabel("Intensidad")
        plt.ylabel("Frecuencia")

    # Ajustar los espacios entre subplots
    plt.tight_layout()
    plt.show()

In [ ]:
for image in imagenes:
    # Convertir a espacio LAB
    lab = cv2.cvtColor(image, cv2.COLOR_BGR2LAB)
    l, a, b = cv2.split(lab)

    # Aplicar ecualización de histograma al canal de luminancia
    l_ecualizado = cv2.equalizeHist(l)

    # Reconstruir la imagen LAB con el canal L ecualizado
    lab_ecualizado = cv2.merge((l_ecualizado, a, b))
    imagen_ecualizada = cv2.cvtColor(lab_ecualizado, cv2.COLOR_LAB2BGR)

    # Aplicar un suavizado (opcional para reducir ruido)
    imagen_ecualizada_suavizada = cv2.bilateralFilter(imagen_ecualizada, d=9, sigmaColor=75, sigmaSpace=75)

    # Configurar la figura y tamaño
    plt.figure(figsize=(14, 12))

    # Imagen original
    plt.subplot(2, 2, 1)
    plt.title("Original")
    plt.imshow(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
    plt.axis('off')  # Eliminar ejes

    # Imagen ecualizada
    plt.subplot(2, 2, 2)
    plt.title("Ecualizada (equalizeHist en LAB)")
    plt.imshow(cv2.cvtColor(imagen_ecualizada, cv2.COLOR_BGR2RGB))
    plt.axis('off')  # Eliminar ejes

    # Imagen ecualizada y suavizada
    plt.subplot(2, 2, 3)
    plt.title("Ecualizada y Suavizada")
    plt.imshow(cv2.cvtColor(imagen_ecualizada_suavizada, cv2.COLOR_BGR2RGB))
    plt.axis('off')  # Eliminar ejes

    # Ajustar los espacios entre subplots
    plt.tight_layout()
    plt.show()



In [ ]:
# Abrimos la imagen con openCV: Se trata de una matriz de numpy
# img = cv2.imread('../images/bananos.jpg')
for i in range(len(imagenes)):
  gray_img = gray_img = cv2.cvtColor(imagenes[i], cv2.COLOR_BGR2GRAY)
  # Aplicar ecualización del histograma
  equalized_image = cv2.equalizeHist(gray_img)

  hist_o,bins_o = np.histogram(gray_img.flatten(),256,[0,256])

  cdf_o = hist_o.cumsum()
  cdf_normalized_o = cdf_o * float(hist_o.max()) / cdf_o.max()

  hist_e,bins_e = np.histogram(equalized_image.flatten(),256,[0,256])

  cdf_e = hist_e.cumsum()
  cdf_normalized_e = cdf_e * float(hist_e.max()) / cdf_e.max()

  # Configurar la figura y tamaño
  plt.figure(figsize=(14, 10))

  # Imagen original
  plt.subplot(2, 2, 1)
  plt.title("Original")
  plt.imshow(gray_img, cmap='gray')
  plt.axis('off')  # Eliminar ejes

  # Imagen ecualizada
  plt.subplot(2, 2, 2)
  plt.title("Ecualizada")
  plt.imshow(equalized_image, cmap='gray')
  plt.axis('off')  # Eliminar ejes

  # Aplanamos matriz original
  vector_g = gray_img.flatten()

  # Histograma de la imagen original
  plt.subplot(2, 2, 3)
  plt.plot(cdf_normalized_o, color = 'b')
  plt.hist(vector_g, bins=256, color='gray', alpha=0.9)
  plt.title("Histograma Escala de Grises")
  plt.xlabel("Intensidad")
  plt.ylabel("Frecuencia")

  # Histograma de la imagen ecualizada
  vector_e = equalized_image.flatten()
  plt.subplot(2, 2, 4)
  plt.plot(cdf_normalized_e, color = 'b')
  plt.hist(vector_e, bins=256, color='gray', alpha=0.9)
  plt.title("Histograma Ecualizado")
  plt.xlabel("Intensidad")
  plt.ylabel("Frecuencia")

  # Ajustar los espacios entre subplots
  plt.tight_layout()
  plt.show()

### CLAHE (Contrast Limited Adaptive Histogram Equalization)
[CLAHE OpenCV](https://docs.opencv.org/4.x/d5/daf/tutorial_py_histogram_equalization.html)

- Divide la imagen en pequeñas regiones llamadas "tiles" y aplica equalización del histograma de forma local.

- Mejora el contraste localmente sin sobrecargarlo globalmente.

- Puede crear efectos de bloques o mosaicos si las transiciones entre tiles no están bien suavizadas.

*Estos métodos pueden combinarse dependiendo de la naturaleza de la imagen. Por ejemplo, combinar CLAHE con corrección gamma suele dar buenos resultados para imágenes nocturnas.*

In [ ]:
img_clahes = []
for i in range(len(imagenes)):
  # Leer la imagen en escala de grises
  img = cv2.cvtColor(imagenes[i], cv2.COLOR_BGR2GRAY)

  # 1. Equalización del histograma
  img_eq = cv2.equalizeHist(img)

  # 2. Crear un objeto CLAHE y aplicar
  clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8, 8))
  img_clahe = clahe.apply(img)
  img_clahes.append(img_clahe)
  # Mostrar imágenes y sus histogramas
  fig, axes = plt.subplots(2, 3, figsize=(20, 10))

  # Imagen original
  axes[0, 0].imshow(img, cmap='gray')
  axes[0, 0].set_title("Imagen Original")
  axes[0, 0].axis('off')
  axes[1, 0].hist(img.ravel(), bins=256, range=(0, 256), color='black')
  axes[1, 0].set_title("Histograma Original")

  # Imagen con equalización del histograma
  axes[0, 1].imshow(img_eq, cmap='gray')
  axes[0, 1].set_title("Equalización del Histograma")
  axes[0, 1].axis('off')
  axes[1, 1].hist(img_eq.ravel(), bins=256, range=(0, 256), color='black')
  axes[1, 1].set_title("Histograma Equalizado")

  # Imagen con CLAHE
  axes[0, 2].imshow(img_clahe, cmap='gray')
  axes[0, 2].set_title("Imagen con CLAHE")
  axes[0, 2].axis('off')
  axes[1, 2].hist(img_clahe.ravel(), bins=256, range=(0, 256), color='black')
  axes[1, 2].set_title("Histograma CLAHE")

  plt.tight_layout()
  plt.show()


## Operaciones Aritméticas

### Resta

In [ ]:
# Generar todas las combinaciones de dos imágenes
combinaciones = list(combinations(range(len(imagenes)), 2))
images1_gray = []
images2_gray = []
# Iterar sobre cada par de combinaciones
for idx1, idx2 in combinaciones:
    print(f"Comparando Imagen {idx1 + 1} con Imagen {idx2 + 1}")

    # Leer las imágenes del par
    image1 = imagenes[idx1]
    image2 = imagenes[idx2]

    # Redimensionar la segunda imagen para que coincida con la primera
    image2_resized = cv2.resize(image2, (image1.shape[1], image1.shape[0]))

    # Realizar la operación de resta en color
    result_color = cv2.absdiff(image1, image2_resized)

    # Convertir a escala de grises
    image1_gray = cv2.cvtColor(image1, cv2.COLOR_BGR2GRAY)
    images1_gray.append(image1_gray)
    image2_resized_gray = cv2.cvtColor(image2_resized, cv2.COLOR_BGR2GRAY)
    images2_gray.append(image2_resized_gray)

    # Realizar la operación de resta en escala de grises
    result_gray = cv2.absdiff(image1_gray, image2_resized_gray)

    # Visualizar los resultados para este par
    plt.figure(figsize=(15, 10))

    # Imágenes en color
    plt.subplot(2, 3, 1)
    plt.title(f"Imagen {idx1 + 1} Color")
    plt.imshow(cv2.cvtColor(image1, cv2.COLOR_BGR2RGB))
    plt.axis("off")

    plt.subplot(2, 3, 2)
    plt.title(f"Imagen {idx2 + 1} Color")
    plt.imshow(cv2.cvtColor(image2_resized, cv2.COLOR_BGR2RGB))
    plt.axis("off")

    plt.subplot(2, 3, 3)
    plt.title("Resultado Resta Color")
    plt.imshow(cv2.cvtColor(result_color, cv2.COLOR_BGR2RGB))
    plt.axis("off")

    # Imágenes en escala de grises
    plt.subplot(2, 3, 4)
    plt.title(f"Imagen {idx1 + 1} Escala de grises")
    plt.imshow(image1_gray, cmap="gray")
    plt.axis("off")

    plt.subplot(2, 3, 5)
    plt.title(f"Imagen {idx2 + 1} Escala de grises")
    plt.imshow(image2_resized_gray, cmap="gray")
    plt.axis("off")

    plt.subplot(2, 3, 6)
    plt.title("Resultado Escala de grises")
    plt.imshow(result_gray, cmap="gray")
    plt.axis("off")

    plt.tight_layout()
    plt.show()

    # Mostrar valores de píxeles (opcional)
    fila, columna = 84, 150
    print(f'Pixel Imagen {idx1 + 1} (Color): {image1[fila, columna]}')
    print(f'Pixel Imagen {idx2 + 1} (Color): {image2_resized[fila, columna]}')
    print(f'Pixel Resultado (Color): {result_color[fila, columna]}')
    print(f'Pixel Imagen {idx1 + 1} (Gris): {image1_gray[fila, columna]}')
    print(f'Pixel Imagen {idx2 + 1} (Gris): {image2_resized_gray[fila, columna]}')
    print(f'Pixel Resultado (Gris): {result_gray[fila, columna]}')


In [ ]:
differences = []
for i in range(len(imagenes)):
  difference = cv2.absdiff(images1_gray[i], images2_gray[i])
  differences.append(difference)
  # Aplicar un umbral para resaltar cambios 50 es el umbral
  _, binary_diff = cv2.threshold(difference, 50, 255, cv2.THRESH_BINARY)

  # Visualizar las imágenes
  plt.figure(figsize=(15, 10))

  plt.subplot(2, 2, 1)
  plt.title("Imagen 1 - Escala de grises")
  plt.imshow(images1_gray[i], cmap="gray")
  plt.axis("off")

  plt.subplot(2, 2, 2)
  plt.title("Imagen 2 - Escala de grises")
  plt.imshow(images2_gray[i], cmap="gray")
  plt.axis("off")

  plt.subplot(2, 2, 3)
  plt.title("Diferencia Absoluta")
  plt.imshow(difference, cmap="gray")
  plt.axis("off")

  plt.subplot(2, 2, 4)
  plt.title("Diferencia Binaria")
  plt.imshow(binary_diff, cmap="gray")
  plt.axis("off")

  plt.show()

In [ ]:
def añadir_ruido_gaussiano(imagen, mean=0, sigma=50):
    ruido = np.random.normal(mean, sigma, imagen.shape).astype(np.float32)
    imagen_ruido = imagen.astype(np.float32) + ruido
    imagen_ruido = np.clip(imagen_ruido, 0, 255).astype(np.uint8)
    return imagen_ruido

In [ ]:
for i in range(len(imagenes)):
  num_iteraciones = 50
  imagenes_con_ruido = []
  imagen_promediada = np.zeros_like(imagenes[i], dtype=np.float32)

  for _ in range(num_iteraciones):
      imagen_con_ruido = añadir_ruido_gaussiano(imagenes[i])
      imagenes_con_ruido.append(imagen_con_ruido)
      imagen_promediada += imagen_con_ruido.astype(np.float32)

  # Promediar las imágenes
  imagen_promediada /= num_iteraciones
  imagen_promediada = imagen_promediada.astype(np.uint8)

  # Visualizar resultados
  plt.figure(figsize=(30, 15))

  # Mostrar una imagen con ruido
  plt.subplot(1, 3, 2)
  plt.title("Imagen con Ruido Gaussiano")
  plt.imshow(imagenes_con_ruido[0])
  plt.axis("off")

  # Mostrar la imagen original
  plt.subplot(1, 3, 1)
  plt.title("Imagen Original")
  plt.imshow(imagenes[i])
  plt.axis("off")

  # Mostrar la imagen promediada
  plt.subplot(1, 3, 3)
  plt.title("Imagen Promediada")
  plt.imshow(imagen_promediada)
  plt.axis("off")

  plt.tight_layout()
  plt.show()

### Multiplicación
- Podemos modificar el brillo aplicando un factor de escala a toda la imagen
- Recuerda que al aplicar un factor de escala hay pixeles que pueden superar el umbral!
- Ej: un pixel con intensidad 254 con una escala de 2 pasará a valer 508. Se puede optar por recorte de valores entre 0 y 255 (8bits) o normalizar

In [ ]:
# Subir una imagen desde local
uploaded = files.upload()
imagenes = []
images_rgb = []

# Cargar la imagen usando cv2
for filename in uploaded.keys():
  image = cv2.imread(filename, cv2.IMREAD_UNCHANGED)  # Cargar sin alterar el formato original
  imagenes.append(image)
for i in range(len(imagenes)):
  if len(imagenes[i].shape) == 2:  # Solo tiene ancho y alto, es escala de grises
      is_grayscale = True
      image_rgb = cv2.cvtColor(imagenes[i], cv2.COLOR_GRAY2RGB)  # Convertir a RGB para visualización
  else:
      is_grayscale = False
      image_rgb = cv2.cvtColor(imagenes[i], cv2.COLOR_BGR2RGB)  # Convertir de BGR a RGB para visualización

# Definir el factor de escala
  factor = 2.5  # Cambia este valor para aumentar o disminuir el brillo (Ejemplo: 0.8 para reducir)

# 1. Transformación con recorte (np.clip)
  if is_grayscale:
      image_clipped = np.clip(imagenes[i] * factor, 0, 255).astype(np.uint8)
      clipped_rgb = cv2.cvtColor(image_clipped, cv2.COLOR_GRAY2RGB)  # Para visualizar
  else:
      image_clipped = np.clip(imagenes[i] * factor, 0, 255).astype(np.uint8)
      clipped_rgb = cv2.cvtColor(image_clipped, cv2.COLOR_BGR2RGB)

# 2. Transformación con normalización
# if is_grayscale:
#     normalized = ((image * factor - np.min(image * factor)) / (np.max(image * factor) - np.min(image * factor)) * 255).astype(np.uint8)
#     normalized_rgb = cv2.cvtColor(normalized, cv2.COLOR_GRAY2RGB)  # Para visualizar
# else:
#     normalized = ((image * factor - np.min(image * factor)) / (np.max(image * factor) - np.min(image * factor)) * 255).astype(np.uint8)
#     normalized_rgb = cv2.cvtColor(normalized, cv2.COLOR_BGR2RGB)

# Mostrar las imágenes lado a lado
  plt.figure(figsize=(18, 6))

  # Imagen original
  plt.subplot(1, 3, 1)
  plt.title("Imagen Original")
  plt.imshow(image_rgb)
  plt.axis("off")

  # Imagen con recorte
  plt.subplot(1, 3, 2)
  plt.title("Transformación con Recorte")
  plt.imshow(clipped_rgb)
  plt.axis("off");

# Imagen con normalización
# plt.subplot(1, 3, 3)
# plt.title("Transformación con Normalización")
# plt.imshow(normalized_rgb)
# plt.axis("off")

# plt.tight_layout()
# plt.show()

#pintar mismo pixel en las 3 imagenes
# fila, columna = 400, 700
# print(f'Pixel Imagen Original: {image[fila, columna]}')
# print(f'Pixel Imagen con Recorte: {image_clipped[fila, columna]}')
# print(f'Pixel Imagen con Normalización: {normalized[fila, columna]}')

Vamos a intentarlo de forma automática

In [ ]:
def mostrar_imagen_grande(imagen, titulo="Imagen"):
    """
    Muestra una imagen en una figura grande.
    """
    plt.figure(figsize=(12, 8))
    plt.imshow(cv2.cvtColor(imagen, cv2.COLOR_BGR2RGB))
    plt.title(titulo, fontsize=16)
    plt.axis('off')
    plt.show()

In [ ]:
def identificar_rangos_densos(histograma, umbral_frecuencia=0.0005, min_bins_consecutivos=2):
    """
    Identifica rangos densos en el histograma basado en la concentración de valores.
    """
    total_pixeles = np.sum(histograma)
    umbral_absoluto = umbral_frecuencia * total_pixeles

    # Bins que cumplen el umbral
    bins_significativos = histograma >= umbral_absoluto

    # Detectar rangos densos
    rangos_densos = []
    inicio = None

    for i, es_significativo in enumerate(bins_significativos):
        if es_significativo:
            if inicio is None:
                inicio = i
        else:
            if inicio is not None:
                if i - inicio >= min_bins_consecutivos:
                    rangos_densos.append((inicio, i - 1))
                inicio = None

    # Verificar si el último rango se cerró
    if inicio is not None and len(histograma) - inicio >= min_bins_consecutivos:
        rangos_densos.append((inicio, len(histograma) - 1))

    return rangos_densos

def aplicar_transformacion_logaritmica_rgb(imagen_rgb, c_log):
    """
    Aplica una transformación logarítmica directamente a cada canal RGB.
    """
    # Separar los canales RGB
    canales = cv2.split(imagen_rgb)

    # Aplicar la transformación logarítmica a cada canal
    canales_transformados = []
    for canal in canales:
        canal_normalizado = canal.astype(np.float32) + 1  # Evitar log(0)
        canal_transformado = c_log * np.log10(1 + canal_normalizado)
        canal_transformado = np.uint8(np.clip(canal_transformado, 0, 255))
        canales_transformados.append(canal_transformado)

    # Reconstruir la imagen RGB con los canales transformados
    imagen_transformada = cv2.merge(canales_transformados)
    return imagen_transformada

def mostrar_comparacion_imagenes(imagen_original, imagen_transformada, titulo_original="Original", titulo_transformada="Transformada"):
    """
    Muestra la imagen original y transformada lado a lado para comparación.
    """
    fig, axes = plt.subplots(1, 2, figsize=(12, 6))

    # Imagen original
    axes[0].imshow(cv2.cvtColor(imagen_original, cv2.COLOR_BGR2RGB))
    axes[0].set_title(titulo_original)
    axes[0].axis('off')

    # Imagen transformada
    axes[1].imshow(cv2.cvtColor(imagen_transformada, cv2.COLOR_BGR2RGB))
    axes[1].set_title(titulo_transformada)
    axes[1].axis('off')

    plt.tight_layout()
    plt.show()

def ajustar_c_dinamicamente(gray_img, imagen_rgb, c_inicial=10, incremento=20, umbral_superior=20, max_iteraciones=10):
    c_actual = c_inicial
    iteracion = 0
    rango_cerca = False  # Inicializamos como False
    ultimo_c_valido = None
    ultimo_limite_inferior = None
    ultimo_limite_superior = None
    mejor_imagen_log = None

    while iteracion < max_iteraciones:
        # Normalizar la imagen y aplicar la transformación logarítmica
        img_normalizada = gray_img.astype(np.float32) + 1  # Evitar log(0)
        c_log = c_actual
        imagen_logaritmica = c_log * np.log10(1 + img_normalizada)
        imagen_logaritmica = np.uint8(np.clip(imagen_logaritmica, 0, 255))

        # Calcular el histograma de la imagen logarítmica
        histograma, bins = np.histogram(imagen_logaritmica.ravel(), bins=256, range=(0, 255))

        # Detectar los rangos densos en el histograma
        rangos_densos = identificar_rangos_densos(histograma, umbral_frecuencia=0.0005, min_bins_consecutivos=2)

        if not rangos_densos:  # Si no hay rangos densos
            print(f"No se encontraron rangos densos en la iteración {iteracion + 1}")
            c_actual += incremento  # Pasamos al siguiente valor de c
            iteracion += 1
            continue  # Continuamos con la siguiente iteración

        # Si hay rangos densos, obtenemos los límites
        limite_inferior = rangos_densos[0][0]
        limite_superior = rangos_densos[-1][1]

        densidad_izquierda = np.sum(histograma[:150])
        densidad_derecha = np.sum(histograma[150:])

        densidad_total = densidad_izquierda + densidad_derecha
        densidad_izquierda_porcentaje = densidad_izquierda / densidad_total
        densidad_derecha_porcentaje = densidad_derecha / densidad_total

        # Crear la figura con dos subgráficos (lado a lado)
        fig, axes = plt.subplots(1, 2, figsize=(12, 6))

        # Mostrar la imagen logarítmica en el primer subgráfico
        axes[0].imshow(imagen_logaritmica, cmap='gray')
        axes[0].set_title(f"Imagen Logarítmica (c = {c_actual})")
        axes[0].axis('off')

        # Mostrar el histograma en el segundo subgráfico
        axes[1].hist(imagen_logaritmica.ravel(), bins=256, range=(0, 255), color='blue', alpha=0.7, label="Histograma General")

        # Resaltar la densidad principal (entre el limite_inferior y limite_superior)
        histograma_resaltado = np.zeros_like(histograma)  # Inicializamos un histograma en blanco
        histograma_resaltado[limite_inferior:limite_superior] = histograma[limite_inferior:limite_superior]
        axes[1].bar(bins[:-1], histograma_resaltado, width=1, color='red', alpha=0.7, label="Rango de Densidad Alta")

        # Añadir líneas para los límites
        axes[1].axvline(x=limite_inferior, color='red', linestyle='--', label=f'Limite Inferior: {limite_inferior}')
        axes[1].axvline(x=limite_superior, color='green', linestyle='--', label=f'Limite Superior: {limite_superior}')

        axes[1].set_title(f"Histograma Logarítmico (c = {c_actual})")
        axes[1].set_xlabel("Intensidad")
        axes[1].set_ylabel("Frecuencia")
        axes[1].legend()

        plt.xlim(0, 255)
        plt.tight_layout()
        plt.show()

        print(f"Iteración {iteracion + 1}: c = {c_actual}")
        print(f"Rango de densidad principal: {limite_inferior} - {limite_superior}")

        # Verificar si el rango está cerca de 150
        esta_cerca = (limite_inferior <= 150) and (limite_superior >= 150)
        print(f"¿El rango está cerca de 150? {'Sí' if esta_cerca else 'No'}")

        if esta_cerca:
            # Guardamos el último valor válido de c y los límites correspondientes
            ultimo_c_valido = c_actual
            ultimo_limite_inferior = limite_inferior
            ultimo_limite_superior = limite_superior
            mejor_imagen_log = imagen_logaritmica  # Guardamos la mejor imagen logarítmica
            rango_cerca = True  # Indicamos que hemos encontrado un valor cerca de 150
        else:
            if rango_cerca:  # Si antes estaba cerca y ahora ya no lo está, terminamos
                print("El rango ya no está cerca de 150, deteniendo la iteración.")
                break

        # Continuamos con la búsqueda si el rango no ha dejado de estar cerca de 150
        c_actual += incremento
        iteracion += 1

    print(f"No se encontró un valor de c que cumpla con el criterio.")

    # Si encontramos un valor válido, mostrar el último encontrado
    if ultimo_c_valido is not None:
        print(f"Último valor válido de c: {ultimo_c_valido}")
        print(f"Rango de densidad principal: {ultimo_limite_inferior} - {ultimo_limite_superior}")

        # Usamos el último valor válido de c para transformar la imagen
        imagen_rgb_transformada = aplicar_transformacion_logaritmica_rgb(imagen_rgb, ultimo_c_valido)
        mostrar_comparacion_imagenes(
            imagen_original=imagen_rgb,
            imagen_transformada=imagen_rgb_transformada,
            titulo_original="Imagen Original",
            titulo_transformada=f"Imagen Transformada (c = {ultimo_c_valido})"
        )
        mostrar_imagen_grande(imagen_rgb_transformada, titulo=f"Imagen Transformada (c = {c_actual})")
        return ultimo_c_valido, mejor_imagen_log, (ultimo_limite_inferior, ultimo_limite_superior)

    return c_actual, mejor_imagen_log, (ultimo_limite_inferior, ultimo_limite_superior)


def aplicar_transformacion_rgb(imagen_rgb, c_log):
    # Separar los canales RGB
    canales = cv2.split(imagen_rgb)

    # Aplicar la transformación logarítmica a cada canal
    canales_transformados = []
    for canal in canales:
        canal_normalizado = canal.astype(np.float32) + 1  # Evitar log(0)
        canal_transformado = c_log * np.log10(1 + canal_normalizado)
        canal_transformado = np.uint8(np.clip(canal_transformado, 0, 255))
        canales_transformados.append(canal_transformado)

    # Reconstruir la imagen RGB con los canales transformados
    imagen_transformada = cv2.merge(canales_transformados)
    return imagen_transformada

"""uploaded = files.upload()

image_path = list(uploaded.keys())[0]
imagenes = []
for filename in uploaded.keys():
    img = cv2.imread(filename)
    imagenes.append(img)"""

gray_imgs = [cv2.cvtColor(img, cv2.COLOR_BGR2GRAY) for img in imagenes]

for i in range(len(gray_imgs)):
  gray_img = gray_imgs[i]
  c_optimo, imagen_log_final, (limite_inferior, limite_superior) = ajustar_c_dinamicamente(
      gray_img, imagenes[i], c_inicial=10, incremento=10, umbral_superior=20, max_iteraciones=10
  )

  print(f"Valor óptimo de c: {c_optimo}")
  print(f"Rango de densidad principal: {limite_inferior} - {limite_superior}")

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import cv2
from google.colab import files

def identificar_rangos_densos(histograma, umbral_frecuencia=0.0005, min_bins_consecutivos=2):
    """
    Identifica rangos densos en el histograma basado en la concentración de valores.
    """
    total_pixeles = np.sum(histograma)
    umbral_absoluto = umbral_frecuencia * total_pixeles

    # Bins que cumplen el umbral
    bins_significativos = histograma >= umbral_absoluto

    # Detectar rangos densos
    rangos_densos = []
    inicio = None

    for i, es_significativo in enumerate(bins_significativos):
        if es_significativo:
            if inicio is None:
                inicio = i
        else:
            if inicio is not None:
                if i - inicio >= min_bins_consecutivos:
                    rangos_densos.append((inicio, i - 1))
                inicio = None

    # Verificar si el último rango se cerró
    if inicio is not None and len(histograma) - inicio >= min_bins_consecutivos:
        rangos_densos.append((inicio, len(histograma) - 1))

    return rangos_densos

def mostrar_imagen_grande(imagen, titulo="Imagen"):
    """
    Muestra una imagen en una figura grande.
    """
    plt.figure(figsize=(12, 8))
    plt.imshow(cv2.cvtColor(imagen, cv2.COLOR_BGR2RGB))
    plt.title(titulo, fontsize=16)
    plt.axis('off')
    plt.show()

def ajustar_c_dinamicamente_rgb(imagen_rgb, c_inicial=10, incremento=20, umbral_superior=20, max_iteraciones=10):
    """
    Ajusta dinámicamente el valor de c para imágenes RGB aplicando la transformación logarítmica
    y detectando rangos densos en el histograma.
    """
    c_actual = c_inicial
    iteracion = 0
    rango_cerca = False  # Inicializamos como False
    ultimo_c_valido = None
    ultimo_limite_inferior = None
    ultimo_limite_superior = None
    mejor_imagen_log = None

    while iteracion < max_iteraciones:
        # Separar los canales RGB y normalizarlos
        canales = cv2.split(imagen_rgb)
        canales_transformados = []
        for canal in canales:
            canal_normalizado = canal.astype(np.float32) + 1  # Evitar log(0)
            canal_transformado = c_actual * np.log10(1 + canal_normalizado)
            canales_transformados.append(canal_transformado)

        # Reconstruir la imagen RGB con los canales transformados
        imagen_logaritmica = cv2.merge([np.uint8(np.clip(canal, 0, 255)) for canal in canales_transformados])

        # Calcular histogramas para cada canal
        histogramas = [np.histogram(canal.ravel(), bins=256, range=(0, 255))[0] for canal in canales_transformados]
        rangos_densos_rgb = [identificar_rangos_densos(histograma) for histograma in histogramas]

        # Unificar límites en los tres canales
        limites_inferiores = [rango[0][0] if rango else None for rango in rangos_densos_rgb]
        limites_superiores = [rango[-1][1] if rango else None for rango in rangos_densos_rgb]

        # Verificar si todos los canales tienen rangos densos válidos
        if None in limites_inferiores or None in limites_superiores:
            print(f"No se encontraron rangos densos en todos los canales para c = {c_actual}")
            c_actual += incremento
            iteracion += 1
            continue

        limite_inferior = min(limites_inferiores)
        limite_superior = max(limites_superiores)

        # Calcular densidades en los canales RGB
        densidades_rgb = [
            (np.sum(histograma[:150]), np.sum(histograma[150:]))
            for histograma in histogramas
        ]
        densidad_total_rgb = [sum(densidad) for densidad in densidades_rgb]
        densidad_porcentajes_rgb = [
            (densidad[0] / total, densidad[1] / total)
            for densidad, total in zip(densidades_rgb, densidad_total_rgb)
        ]

        # Crear la figura con subgráficos (lado a lado)
        fig, axes = plt.subplots(1, 4, figsize=(16, 6))

        # Mostrar la imagen logarítmica
        axes[0].imshow(cv2.cvtColor(imagen_logaritmica, cv2.COLOR_BGR2RGB))
        axes[0].set_title(f"Imagen Transformada (c = {c_actual})")
        axes[0].axis('off')

        # Mostrar los histogramas para los canales R, G, B
        colores = ['red', 'green', 'blue']
        for i, (histograma, color, ax) in enumerate(zip(histogramas, colores, axes[1:])):
            ax.bar(range(256), histograma, color=color, alpha=0.7)
            ax.axvline(x=limites_inferiores[i], color='black', linestyle='--', label='Límite Inferior')
            ax.axvline(x=limites_superiores[i], color='black', linestyle='--', label='Límite Superior')
            ax.set_title(f"Histograma {color.upper()}")
            ax.set_xlim(0, 255)
            ax.legend()

        plt.tight_layout()
        plt.show()

        print(f"Iteración {iteracion + 1}: c = {c_actual}")
        print(f"Rango de densidad principal RGB: {limite_inferior} - {limite_superior}")

        # Verificar si el rango está cerca de 150
        esta_cerca = all(limite_inferior < 150 < limite_superior for limite_inferior, limite_superior in zip(limites_inferiores, limites_superiores))
        print(f"¿El rango está cerca de 150? {'Sí' if esta_cerca else 'No'}")

        if esta_cerca:
            ultimo_c_valido = c_actual
            ultimo_limite_inferior = limite_inferior
            ultimo_limite_superior = limite_superior
            mejor_imagen_log = imagen_logaritmica
            rango_cerca = True
        else:
            if rango_cerca:
                print("El rango ya no está cerca de 150, deteniendo la iteración.")
                break

        c_actual += incremento
        iteracion += 1

    if ultimo_c_valido is not None:
        print(f"Último valor válido de c: {ultimo_c_valido}")
        print(f"Rango de densidad principal RGB: {ultimo_limite_inferior} - {ultimo_limite_superior}")

        mostrar_imagen_grande(imagen_logaritmica, titulo=f"Imagen Transformada (c = {c_actual})")

        return ultimo_c_valido, mejor_imagen_log, (ultimo_limite_inferior, ultimo_limite_superior)

    print("No se encontró un valor de c que cumpla con el criterio.")
    return c_actual, mejor_imagen_log, (ultimo_limite_inferior, ultimo_limite_superior)



# Cargar imágenes
uploaded = files.upload()

imagenes = []
for filename in uploaded.keys():
    img = cv2.imread(filename)
    imagenes.append(img)

# Aplicar ajuste dinámico de c en imágenes RGB
for i, img_rgb in enumerate(imagenes):
    c_optimo, imagen_log_final, (limite_inferior, limite_superior) = ajustar_c_dinamicamente_rgb(
        img_rgb, c_inicial=10, incremento=10, umbral_superior=20, max_iteraciones=10
    )

    print(f"Valor óptimo de c: {c_optimo}")
    print(f"Rango de densidad principal RGB: {limite_inferior} - {limite_superior}")


### Lo mismo con skimage

In [ ]:
from skimage import exposure

def identificar_rangos_densos(histograma, umbral_frecuencia=0.0005, min_bins_consecutivos=2):
    """
    Identifica rangos densos en el histograma basado en la concentración de valores.
    """
    total_pixeles = np.sum(histograma)
    umbral_absoluto = umbral_frecuencia * total_pixeles

    # Bins que cumplen el umbral
    bins_significativos = histograma >= umbral_absoluto

    # Detectar rangos densos
    rangos_densos = []
    inicio = None

    for i, es_significativo in enumerate(bins_significativos):
        if es_significativo:
            if inicio is None:
                inicio = i
        else:
            if inicio is not None:
                if i - inicio >= min_bins_consecutivos:
                    rangos_densos.append((inicio, i - 1))
                inicio = None

    # Verificar si el último rango se cerró
    if inicio is not None and len(histograma) - inicio >= min_bins_consecutivos:
        rangos_densos.append((inicio, len(histograma) - 1))

    return rangos_densos

def aplicar_transformacion_logaritmica_rgb(imagen_rgb, c_log):
    """
    Aplica una transformación logarítmica directamente a cada canal RGB.
    """
    # Separar los canales RGB
    canales = cv2.split(imagen_rgb)

    # Aplicar la transformación logarítmica a cada canal
    canales_transformados = []
    for canal in canales:
        canal_normalizado = canal.astype(np.float32) + 1  # Evitar log(0)
        canal_transformado = exposure.adjust_log(canal_normalizado, c_log)
        canal_transformado = np.uint8(np.clip(canal_transformado, 0, 255))
        canales_transformados.append(canal_transformado)

    # Reconstruir la imagen RGB con los canales transformados
    imagen_transformada = cv2.merge(canales_transformados)
    return imagen_transformada

def mostrar_comparacion_imagenes(imagen_original, imagen_transformada, titulo_original="Original", titulo_transformada="Transformada"):
    """
    Muestra la imagen original y transformada lado a lado para comparación.
    """
    fig, axes = plt.subplots(1, 2, figsize=(12, 6))

    # Imagen original
    axes[0].imshow(cv2.cvtColor(imagen_original, cv2.COLOR_BGR2RGB))
    axes[0].set_title(titulo_original)
    axes[0].axis('off')

    # Imagen transformada
    axes[1].imshow(cv2.cvtColor(imagen_transformada, cv2.COLOR_BGR2RGB))
    axes[1].set_title(titulo_transformada)
    axes[1].axis('off')

    plt.tight_layout()
    plt.show()

def ajustar_c_dinamicamente(gray_img, imagen_rgb, c_inicial=10, incremento=20, umbral_superior=20, max_iteraciones=10):
    c_actual = c_inicial
    iteracion = 0
    rango_cerca = False  # Inicializamos como False
    ultimo_c_valido = None
    ultimo_limite_inferior = None
    ultimo_limite_superior = None
    mejor_imagen_log = None

    while iteracion < max_iteraciones:
        # Normalizar la imagen y aplicar la transformación logarítmica
        img_normalizada = gray_img.astype(np.float32) + 1  # Evitar log(0)
        c_log = c_actual
        imagen_logaritmica = exposure.adjust_log(img_normalizada, c_log)
        imagen_logaritmica = np.uint8(np.clip(imagen_logaritmica, 0, 255))

        # Calcular el histograma de la imagen logarítmica
        histograma, bins = np.histogram(imagen_logaritmica.ravel(), bins=256, range=(0, 255))

        # Detectar los rangos densos en el histograma
        rangos_densos = identificar_rangos_densos(histograma, umbral_frecuencia=0.0005, min_bins_consecutivos=2)

        if not rangos_densos:  # Si no hay rangos densos
            print(f"No se encontraron rangos densos en la iteración {iteracion + 1}")
            c_actual += incremento  # Pasamos al siguiente valor de c
            iteracion += 1
            continue  # Continuamos con la siguiente iteración

        # Si hay rangos densos, obtenemos los límites
        limite_inferior = rangos_densos[0][0]
        limite_superior = rangos_densos[-1][1]

        densidad_izquierda = np.sum(histograma[:150])
        densidad_derecha = np.sum(histograma[150:])

        densidad_total = densidad_izquierda + densidad_derecha
        densidad_izquierda_porcentaje = densidad_izquierda / densidad_total
        densidad_derecha_porcentaje = densidad_derecha / densidad_total

        # Crear la figura con dos subgráficos (lado a lado)
        fig, axes = plt.subplots(1, 2, figsize=(12, 6))

        # Mostrar la imagen logarítmica en el primer subgráfico
        axes[0].imshow(imagen_logaritmica, cmap='gray')
        axes[0].set_title(f"Imagen Logarítmica (c = {c_actual})")
        axes[0].axis('off')

        # Mostrar el histograma en el segundo subgráfico
        axes[1].hist(imagen_logaritmica.ravel(), bins=256, range=(0, 255), color='blue', alpha=0.7, label="Histograma General")

        # Resaltar la densidad principal (entre el limite_inferior y limite_superior)
        histograma_resaltado = np.zeros_like(histograma)  # Inicializamos un histograma en blanco
        histograma_resaltado[limite_inferior:limite_superior] = histograma[limite_inferior:limite_superior]
        axes[1].bar(bins[:-1], histograma_resaltado, width=1, color='red', alpha=0.7, label="Rango de Densidad Alta")

        # Añadir líneas para los límites
        axes[1].axvline(x=limite_inferior, color='red', linestyle='--', label=f'Limite Inferior: {limite_inferior}')
        axes[1].axvline(x=limite_superior, color='green', linestyle='--', label=f'Limite Superior: {limite_superior}')

        axes[1].set_title(f"Histograma Logarítmico (c = {c_actual})")
        axes[1].set_xlabel("Intensidad")
        axes[1].set_ylabel("Frecuencia")
        axes[1].legend()

        plt.xlim(0, 255)
        plt.tight_layout()
        plt.show()

        print(f"Iteración {iteracion + 1}: c = {c_actual}")
        print(f"Rango de densidad principal: {limite_inferior} - {limite_superior}")

        # Verificar si el rango está cerca de 150
        esta_cerca = (limite_inferior <= 150) and (limite_superior >= 150)
        print(f"¿El rango está cerca de 150? {'Sí' if esta_cerca else 'No'}")

        if esta_cerca:
            # Guardamos el último valor válido de c y los límites correspondientes
            ultimo_c_valido = c_actual
            ultimo_limite_inferior = limite_inferior
            ultimo_limite_superior = limite_superior
            mejor_imagen_log = imagen_logaritmica  # Guardamos la mejor imagen logarítmica
            rango_cerca = True  # Indicamos que hemos encontrado un valor cerca de 150
        else:
            if rango_cerca:  # Si antes estaba cerca y ahora ya no lo está, terminamos
                print("El rango ya no está cerca de 150, deteniendo la iteración.")
                break

        # Continuamos con la búsqueda si el rango no ha dejado de estar cerca de 150
        c_actual += incremento
        iteracion += 1

    print(f"No se encontró un valor de c que cumpla con el criterio.")

    # Si encontramos un valor válido, mostrar el último encontrado
    if ultimo_c_valido is not None:
        print(f"Último valor válido de c: {ultimo_c_valido}")
        print(f"Rango de densidad principal: {ultimo_limite_inferior} - {ultimo_limite_superior}")

        # Usamos el último valor válido de c para transformar la imagen
        imagen_rgb_transformada = aplicar_transformacion_logaritmica_rgb(imagen_rgb, ultimo_c_valido)
        mostrar_comparacion_imagenes(
            imagen_original=imagen_rgb,
            imagen_transformada=imagen_rgb_transformada,
            titulo_original="Imagen Original",
            titulo_transformada=f"Imagen Transformada (c = {ultimo_c_valido})"
        )
        return ultimo_c_valido, mejor_imagen_log, (ultimo_limite_inferior, ultimo_limite_superior)


    return c_actual, mejor_imagen_log, (ultimo_limite_inferior, ultimo_limite_superior)


uploaded = files.upload()

image_path = list(uploaded.keys())[0]
imagenes = []
for filename in uploaded.keys():
    img = cv2.imread(filename)
    imagenes.append(img)

gray_imgs = [cv2.cvtColor(img, cv2.COLOR_BGR2GRAY) for img in imagenes]

for i in range(len(gray_imgs)):
  gray_img = gray_imgs[i]
  c_optimo, imagen_log_final, (limite_inferior, limite_superior) = ajustar_c_dinamicamente(
      gray_img, imagenes[i], c_inicial=10, incremento=10, umbral_superior=20, max_iteraciones=10
  )

  print(f"Valor óptimo de c: {c_optimo}")
  print(f"Rango de densidad principal: {limite_inferior} - {limite_superior}")

## Gamma

In [ ]:
def identificar_rangos_densos(histograma, umbral_frecuencia=0.0005, min_bins_consecutivos=2):
    """
    Identifica rangos densos en el histograma basado en la concentración de valores.
    """
    total_pixeles = np.sum(histograma)
    umbral_absoluto = umbral_frecuencia * total_pixeles

    # Bins que cumplen el umbral
    bins_significativos = histograma >= umbral_absoluto

    # Detectar rangos densos
    rangos_densos = []
    inicio = None

    for i, es_significativo in enumerate(bins_significativos):
        if es_significativo:
            if inicio is None:
                inicio = i
        else:
            if inicio is not None:
                if i - inicio >= min_bins_consecutivos:
                    rangos_densos.append((inicio, i - 1))
                inicio = None

    # Verificar si el último rango se cerró
    if inicio is not None and len(histograma) - inicio >= min_bins_consecutivos:
        rangos_densos.append((inicio, len(histograma) - 1))

    return rangos_densos

def mostrar_comparacion_imagenes(imagen_original, imagen_transformada, titulo_original="Original", titulo_transformada="Transformada"):
    """
    Muestra la imagen original y transformada lado a lado para comparación.
    """
    fig, axes = plt.subplots(1, 2, figsize=(12, 6))

    # Imagen original
    axes[0].imshow(cv2.cvtColor(imagen_original, cv2.COLOR_BGR2RGB))
    axes[0].set_title(titulo_original)
    axes[0].axis('off')

    # Imagen transformada
    axes[1].imshow(cv2.cvtColor(imagen_transformada, cv2.COLOR_BGR2RGB))
    axes[1].set_title(titulo_transformada)
    axes[1].axis('off')

    plt.tight_layout()
    plt.show()

def ajustar_gamma_dinamicamente(imagen_rgb, c_inicial=1, decremento=0.05, umbral_superior=20, max_iteraciones=10):
    gamma_actual = 1.0  # Valor inicial de gamma
    c_actual = c_inicial
    iteracion = 0
    rango_cerca = False  # Inicializamos como False
    ultimo_c_valido = None
    ultimo_limite_inferior = None
    ultimo_limite_superior = None
    mejor_imagen_gamma = None
    mejor_gamma = gamma_actual

    while iteracion < max_iteraciones:  # Se detiene cuando gamma sea <= 0
        canales = cv2.split(imagen_rgb)

        # Aplicar la transformación logarítmica a cada canal
        canales_transformados = []
        for canal in canales:
            canal_normalizado = canal.astype(np.float32) + 1  # Evitar log(0)
            canal_transformado = c_actual * np.power(canal_normalizado / 255.0, gamma_actual)  # Aplicamos la transformación gamma
            canal_transformado = np.uint8(np.clip(canal_transformado * 255, 0, 255))  # Reescalamos a [0, 255]
            canales_transformados.append(canal_transformado)

        # Reconstruir la imagen RGB con los canales transformados
        imagen_gamma = cv2.merge(canales_transformados)

        histogramas = [np.histogram(canal.ravel(), bins=256, range=(0, 255))[0] for canal in canales_transformados]
        rangos_densos_rgb = [identificar_rangos_densos(histograma) for histograma in histogramas]

        limites_inferiores = [rango[0][0] if rango else None for rango in rangos_densos_rgb]
        limites_superiores = [rango[-1][1] if rango else None for rango in rangos_densos_rgb]

        if None in limites_inferiores or None in limites_superiores:
            print(f"No se encontraron rangos densos en la iteración {iteracion + 1}")
            #gamma_actual -= decremento  # Disminuimos gamma para aclarar la imagen
            #iteracion += 1
            gamma_actual += decremento
            mostrar_imagen_grande(imagen_rgb_transformada, titulo=f"Imagen Transformada (γ = {ultimo_c_valido}, c = {mejor_gamma})")
            continue  # Continuamos con la siguiente iteración

        # Si hay rangos densos, obtenemos los límites
        limite_inferior = min(limites_inferiores)
        limite_superior = max(limites_superiores)

        # Crear la figura con dos subgráficos (lado a lado)
        fig, axes = plt.subplots(1, 4, figsize=(16, 6))

        axes[0].imshow(cv2.cvtColor(imagen_gamma, cv2.COLOR_BGR2RGB))
        axes[0].set_title(f"Imagen Gamma (γ = {gamma_actual}, c = {c_actual})")
        axes[0].axis('off')

        # Mostrar los histogramas para los canales R, G, B
        colores = ['red', 'green', 'blue']
        for i, (histograma, color, ax) in enumerate(zip(histogramas, colores, axes[1:])):
            ax.bar(range(256), histograma, color=color, alpha=0.7)
            ax.axvline(x=limites_inferiores[i], color='black', linestyle='--', label='Límite Inferior')
            ax.axvline(x=limites_superiores[i], color='black', linestyle='--', label='Límite Superior')
            ax.set_title(f"Histograma {color.upper()}")
            ax.set_xlim(0, 255)
            ax.legend()

        plt.tight_layout()
        plt.show()

        print(f"Iteración {iteracion + 1}: γ = {gamma_actual}, c = {c_actual}")
        print(f"Rango de densidad principal: {limite_inferior} - {limite_superior}")

        # Verificar si el rango está cerca de 150
        esta_cerca = (limite_inferior <= 150) and (limite_superior >= 150)
        print(f"¿El rango está cerca de 150? {'Sí' if esta_cerca else 'No'}")

        if esta_cerca:
            # Guardamos el último valor válido de gamma y los límites correspondientes
            ultimo_c_valido = c_actual
            mejor_gamma = gamma_actual
            ultimo_limite_inferior = limite_inferior
            ultimo_limite_superior = limite_superior
            mejor_imagen_gamma = imagen_gamma  # Guardamos la mejor imagen gamma
            rango_cerca = True  # Indicamos que hemos encontrado un valor cerca de 150
            break  # Detenemos la iteración si hemos encontrado un buen valor de gamma

        # Si gamma llegó a un valor mínimo y no cumplió con el criterio, lo detenemos
        if gamma_actual <= 0.1:
            print("El valor de γ es demasiado bajo, deteniendo la iteración.")
            break  # Terminamos si el valor de gamma se acerca demasiado a 0

        # Continuamos con la búsqueda si el rango no ha dejado de estar cerca de 150
        gamma_actual -= decremento  # Disminuimos gamma para aclarar
        iteracion += 1

    # Si encontramos un valor válido, mostrar el último encontrado
    if ultimo_c_valido is not None:
        print(f"Último valor válido de γ: {ultimo_c_valido}")  # Mostramos el último gamma válido
        print(f"Rango de densidad principal: {ultimo_limite_inferior} - {ultimo_limite_superior}")

        # Usamos el último valor válido de γ para transformar la imagen
        imagen_rgb_transformada = aplicar_transformacion_gamma_rgb(imagen_rgb, ultimo_c_valido, mejor_gamma)
        mostrar_comparacion_imagenes(
            imagen_original=imagen_rgb,
            imagen_transformada=imagen_rgb_transformada,
            titulo_original="Imagen Original",
            titulo_transformada=f"Imagen Transformada (γ = {ultimo_c_valido}, c = {mejor_gamma})"
        )
        mostrar_imagen_grande(imagen_rgb_transformada, titulo=f"Imagen Transformada (γ = {ultimo_c_valido}, c = {mejor_gamma})")
        return ultimo_c_valido, mejor_imagen_gamma, (ultimo_limite_inferior, ultimo_limite_superior)

    mostrar_imagen_grande(imagen_rgb_transformada, titulo=f"Imagen Transformada (γ = {ultimo_c_valido}, c = {mejor_gamma})")
    return gamma_actual, mejor_imagen_gamma, (ultimo_limite_inferior, ultimo_limite_superior)



def aplicar_transformacion_rgb(imagen_rgb, c_log):
    # Separar los canales RGB
    canales = cv2.split(imagen_rgb)

    # Aplicar la transformación logarítmica a cada canal
    canales_transformados = []
    for canal in canales:
        canal_normalizado = canal.astype(np.float32) + 1  # Evitar log(0)
        canal_transformado = c_log * np.log10(1 + canal_normalizado)
        canal_transformado = np.uint8(np.clip(canal_transformado, 0, 255))
        canales_transformados.append(canal_transformado)

    # Reconstruir la imagen RGB con los canales transformados
    imagen_transformada = cv2.merge(canales_transformados)
    return imagen_transformada

def mostrar_imagen_grande(imagen, titulo="Imagen"):
    """
    Muestra una imagen en una figura grande.
    """
    plt.figure(figsize=(12, 8))
    plt.imshow(cv2.cvtColor(imagen, cv2.COLOR_BGR2RGB))
    plt.title(titulo, fontsize=16)
    plt.axis('off')
    plt.show()

uploaded = files.upload()

image_path = list(uploaded.keys())[0]
imagenes = []
for filename in uploaded.keys():
    img = cv2.imread(filename)
    imagenes.append(img)

gray_imgs = [cv2.cvtColor(img, cv2.COLOR_BGR2GRAY) for img in imagenes]

for i in range(len(gray_imgs)):
  gray_img = gray_imgs[i]
  c_optimo, imagen_gamma_final, (limite_inferior, limite_superior) = ajustar_gamma_dinamicamente(
        imagenes[i], c_inicial=1, decremento=0.2, umbral_superior=20, max_iteraciones=10
  )

print(f"Valor óptimo de γ: {c_optimo}")
print(f"Rango de densidad principal: {limite_inferior} - {limite_superior}")

# Ajuste en la intensidad


## Análisis del histograma

### Centroides de Intensidad

In [ ]:

def calcular_gamma_y_c_por_percentiles(imagen, percentil_inferior=5, percentil_superior=95):
    """
    Calcula el valor de gamma y c basado en los percentiles de una imagen (grises o RGB).
    """
    if len(imagen.shape) == 2:  # Imagen en escala de grises
        p_inf, p_sup = np.percentile(imagen, [percentil_inferior, percentil_superior])
        rango = p_sup - p_inf

        # Calcular gamma dinámicamente según el rango
        if rango < 100:
            gamma = 0.5  # Aclara
        elif rango > 150:
            gamma = 1.5  # Oscurece
        else:
            gamma = 1.0  # Neutral

        c = 255 / (255 ** gamma)  # Ajuste dinámico para normalización
        return gamma, c

    elif len(imagen.shape) == 3:  # Imagen RGB
        gamma_c_por_canal = {}
        for canal_idx, canal_nombre in enumerate(['Rojo', 'Verde', 'Azul']):
            canal = imagen[:, :, canal_idx]
            p_inf, p_sup = np.percentile(canal, [percentil_inferior, percentil_superior])
            rango = p_sup - p_inf

            if rango < 100:
                gamma = 0.5
            elif rango > 150:
                gamma = 1.5
            else:
                gamma = 1.0

            c = 255 / (255 ** gamma)
            gamma_c_por_canal[canal_nombre] = (gamma, c)
        return gamma_c_por_canal

def aplicar_transformacion_gamma(imagen, gamma, c):
    """
    Aplica la transformación gamma a una imagen (grises o RGB).
    """
    if len(imagen.shape) == 2:  # Imagen en escala de grises
        imagen_normalizada = imagen.astype(np.float32)
        imagen_transformada = c * (imagen_normalizada ** gamma)
        return np.uint8(np.clip(imagen_transformada, 0, 255))

    elif len(imagen.shape) == 3:  # Imagen RGB
        imagen_transformada = np.zeros_like(imagen, dtype=np.float32)
        for canal_idx, canal_nombre in enumerate(['Rojo', 'Verde', 'Azul']):
            canal = imagen[:, :, canal_idx].astype(np.float32)
            imagen_transformada[:, :, canal_idx] = c[canal_nombre] * (canal ** gamma[canal_nombre])
        return np.uint8(np.clip(imagen_transformada, 0, 255))

def mostrar_comparacion_imagenes(imagen_original, imagen_transformada, titulo_original="Original", titulo_transformada="Transformada"):
    """
    Muestra la imagen original y transformada lado a lado para comparación.
    """
    fig, axes = plt.subplots(1, 2, figsize=(12, 6))
    axes[0].imshow(imagen_original, cmap='gray' if len(imagen_original.shape) == 2 else None)
    axes[0].set_title(titulo_original)
    axes[0].axis('off')

    axes[1].imshow(imagen_transformada, cmap='gray' if len(imagen_transformada.shape) == 2 else None)
    axes[1].set_title(titulo_transformada)
    axes[1].axis('off')

    plt.tight_layout()
    plt.show()

# Cargar imágenes
uploaded = files.upload()

# Procesar cada imagen cargada
for filename, file_content in uploaded.items():
    # Leer la imagen en BGR y convertir a RGB
    img_color_bgr = cv2.imdecode(np.frombuffer(file_content, np.uint8), cv2.IMREAD_COLOR)
    img_color_rgb = cv2.cvtColor(img_color_bgr, cv2.COLOR_BGR2RGB)
    img_gray = cv2.cvtColor(img_color_bgr, cv2.COLOR_BGR2GRAY)

    # Calcular gamma y c para la imagen en escala de grises
    gamma_gray, c_gray = calcular_gamma_y_c_por_percentiles(img_gray)
    print(f"Imagen: {filename} (Gris) - Gamma: {gamma_gray:.2f}, c: {c_gray:.2f}")

    # Aplicar transformación gamma a escala de grises
    img_gray_transformada = aplicar_transformacion_gamma(img_gray, gamma_gray, c_gray)
    mostrar_comparacion_imagenes(
        img_gray, img_gray_transformada,
        titulo_original="Imagen Original (Escala de Grises)",
        titulo_transformada="Imagen Transformada (Escala de Grises)"
    )

    # Calcular gamma y c para la imagen RGB
    gamma_c_rgb = calcular_gamma_y_c_por_percentiles(img_color_rgb)
    print(f"Imagen: {filename} (RGB) - Valores de Gamma y c por canal:")
    for canal, (gamma, c) in gamma_c_rgb.items():
        print(f"{canal} - Gamma: {gamma:.2f}, c: {c:.2f}")

    # Aplicar transformación gamma a la imagen RGB
    gamma_rgb = {canal: gamma for canal, (gamma, c) in gamma_c_rgb.items()}
    c_rgb = {canal: c for canal, (gamma, c) in gamma_c_rgb.items()}
    img_rgb_transformada = aplicar_transformacion_gamma(img_color_rgb, gamma_rgb, c_rgb)

    # Mostrar comparación para la imagen RGB
    mostrar_comparacion_imagenes(
        img_color_rgb, img_rgb_transformada,
        titulo_original="Imagen Original (RGB)",
        titulo_transformada="Imagen Transformada (RGB)"
    )


### Percentiles

In [ ]:
import numpy as np
import cv2
import matplotlib.pyplot as plt
from google.colab import files

def ajustar_por_percentiles(imagen, percentil_inferior=5, percentil_superior=95):
    """
    Ajusta una imagen (grises o RGB) utilizando los percentiles especificados.
    """
    if len(imagen.shape) == 2:  # Imagen en escala de grises
        p_inf, p_sup = np.percentile(imagen, [percentil_inferior, percentil_superior])
        imagen_ajustada = np.clip((imagen - p_inf) / (p_sup - p_inf) * 255, 0, 255).astype(np.uint8)
        return imagen_ajustada

    elif len(imagen.shape) == 3:  # Imagen RGB
        imagen_ajustada = np.zeros_like(imagen, dtype=np.uint8)
        for canal_idx in range(3):
            canal = imagen[:, :, canal_idx]
            p_inf, p_sup = np.percentile(canal, [percentil_inferior, percentil_superior])
            imagen_ajustada[:, :, canal_idx] = np.clip((canal - p_inf) / (p_sup - p_inf) * 255, 0, 255).astype(np.uint8)
        return imagen_ajustada

def mostrar_comparacion_imagenes(imagen_original, imagen_transformada, titulo_original="Original", titulo_transformada="Transformada"):
    """
    Muestra la imagen original y transformada lado a lado para comparación.
    """
    fig, axes = plt.subplots(1, 2, figsize=(12, 6))
    axes[0].imshow(imagen_original, cmap='gray' if len(imagen_original.shape) == 2 else None)
    axes[0].set_title(titulo_original)
    axes[0].axis('off')

    axes[1].imshow(imagen_transformada, cmap='gray' if len(imagen_transformada.shape) == 2 else None)
    axes[1].set_title(titulo_transformada)
    axes[1].axis('off')

    plt.tight_layout()
    plt.show()

# Cargar imágenes
uploaded = files.upload()

# Procesar cada imagen cargada
for filename, file_content in uploaded.items():
    # Leer la imagen en BGR y convertir a RGB
    img_color_bgr = cv2.imdecode(np.frombuffer(file_content, np.uint8), cv2.IMREAD_COLOR)
    img_color_rgb = cv2.cvtColor(img_color_bgr, cv2.COLOR_BGR2RGB)
    img_gray = cv2.cvtColor(img_color_bgr, cv2.COLOR_BGR2GRAY)

    # Ajustar la imagen en escala de grises usando percentiles
    img_gray_ajustada = ajustar_por_percentiles(img_gray, percentil_inferior=5, percentil_superior=95)
    print(f"Imagen: {filename} (Gris) - Ajustada por Percentiles")
    mostrar_comparacion_imagenes(
        img_gray, img_gray_ajustada,
        titulo_original="Imagen Original (Escala de Grises)",
        titulo_transformada="Imagen Ajustada (Escala de Grises)"
    )

    # Ajustar la imagen RGB usando percentiles
    img_rgb_ajustada = ajustar_por_percentiles(img_color_rgb, percentil_inferior=5, percentil_superior=95)
    print(f"Imagen: {filename} (RGB) - Ajustada por Percentiles")
    mostrar_comparacion_imagenes(
        img_color_rgb, img_rgb_ajustada,
        titulo_original="Imagen Original (RGB)",
        titulo_transformada="Imagen Ajustada (RGB)"
    )


In [ ]:
uploaded = files.upload()

image_path = list(uploaded.keys())[0]
imagenes = []
for filename in uploaded.keys():
    img = cv2.imread(filename)
    imagenes.append(img)

gray_imgs = [cv2.cvtColor(img, cv2.COLOR_BGR2GRAY) for img in imagenes]


# Procesamiento del histograma

In [ ]:
rgb_imgs = []
gray_imgs = []
for i in range(len(imagenes)):
  rgb_img = cv2.cvtColor(imagenes[i], cv2.COLOR_BGR2RGB)
  rgb_imgs.append(rgb_img)
  plt.figure(figsize=(12, 7))
  plt.subplot(2, 2, 1)
  plt.title("Imagen Original")
  plt.imshow(rgb_img)
  plt.axis('off');
  # Pasamos a Escala de Grises (Un solo valor por píxel donde 0 es negro y 255 blanco)
  gray_img = cv2.cvtColor(imagenes[i], cv2.COLOR_BGR2GRAY)
  gray_imgs.append(gray_img)
  plt.subplot(2, 2, 2)
  plt.title("Imagen en Escala de Grises")
  plt.imshow(gray_img, cmap='gray')
  plt.axis('off');
  plt.subplot(2, 2, 3)
  color = ('r','g','b')
  for i, c in enumerate(color):
      canal = rgb_img[:,:,i].flatten()

      plt.hist(canal, bins=256, color=c, alpha=0.8)
      plt.title("Histograma RGB")
      plt.xlabel("Valor RGB")
      plt.ylabel("Frecuencia");
  # Aplanamos matriz
  vector = gray_img.flatten()
  # Histograma
  plt.subplot(2, 2, 4)
  plt.hist(vector, bins=256)
  plt.title("Histograma Escala de Grises")
  plt.xlabel("Intensidad")
  plt.ylabel("Frecuencia");

In [ ]:
for i in range(len(imagenes)):

    # Aplicar ecualización a la imagen RGB canal por canal
    h, s, v = cv2.split ( imagenes[i] )
    v_equalized = cv2.equalizeHist ( v )
    h_equalized = cv2.equalizeHist ( h )
    s_equalized = cv2.equalizeHist ( s )
    equalized_image_rgb = cv2.merge ( [ h_equalized, s_equalized, v_equalized ])

    # Configurar la figura y tamaño
    plt.figure(figsize=(14, 12))

    # Imagen original en RGB
    plt.subplot(3, 2, 5)
    plt.title("Original (RGB)")
    plt.imshow(cv2.cvtColor(imagenes[i], cv2.COLOR_BGR2RGB))
    plt.axis('off')

    # Imagen ecualizada en RGB
    plt.subplot(3, 2, 6)
    plt.title("Ecualizada (RGB)")
    plt.imshow(cv2.cvtColor(equalized_image_rgb, cv2.COLOR_BGR2RGB))
    plt.axis('off')

    # Ajustar los espacios entre subplots
    plt.tight_layout()
    plt.show()


## Ecualización CLAHE

In [ ]:
for i in range(len(imagenes)):
    # Ecualizar la imagen en escala de grises
    clahe = cv2.createCLAHE(clipLimit=5.0, tileGridSize=(8, 8))

    # Aplicar ecualización a la imagen RGB canal por canal
    canales_rgb = cv2.split(imagenes[i])
    canales_ecualizados = [clahe.apply(canal) for canal in canales_rgb]
    equalized_image_rgb = cv2.merge(canales_ecualizados)

    # Configurar la figura y tamaño
    plt.figure(figsize=(14, 12))

    # Imagen original en RGB
    plt.subplot(3, 2, 5)
    plt.title("Original (RGB)")
    plt.imshow(cv2.cvtColor(imagenes[i], cv2.COLOR_BGR2RGB))
    plt.axis('off')

    # Imagen ecualizada en RGB
    plt.subplot(3, 2, 6)
    plt.title("CLAHE (RGB)")
    plt.imshow(cv2.cvtColor(equalized_image_rgb, cv2.COLOR_BGR2RGB))
    plt.axis('off')

    # Ajustar los espacios entre subplots
    plt.tight_layout()
    plt.show()


## Clahe con rescalar las intensidades

In [ ]:
def optimizar_clahe(imagen, rango_cliplimit=(2.0, 10.0), tamanos_tiles=[(4, 4), (8, 8), (16, 16)]):
    mejor_calidad = -float('inf')  # Inicializamos una métrica de calidad
    mejor_cliplimit = None
    mejor_tilegrid = None
    mejor_imagen = None

    for clip_limit in np.arange(rango_cliplimit[0], rango_cliplimit[1], 0.5):
        for tile_size in tamanos_tiles:
            # Crear el objeto CLAHE con los parámetros actuales
            clahe = cv2.createCLAHE(clipLimit=clip_limit, tileGridSize=tile_size)
            canales_rgb = cv2.split(imagen)
            canales_ecualizados = [clahe.apply(canal) for canal in canales_rgb]
            imagen_clahe = cv2.merge(canales_ecualizados)
            imagen_clahe_reescalada = reescalar_imagen(imagen_clahe)

            # Evaluar la calidad del resultado (puedes personalizar esta métrica)
            histograma = cv2.calcHist([cv2.cvtColor(imagen_clahe_reescalada, cv2.COLOR_BGR2GRAY)], [0], None, [256], [0, 256])
            contraste = np.std(histograma)  # Ejemplo: usar la desviación estándar como métrica de contraste

            if contraste > mejor_calidad:  # Si encontramos una mejor calidad
                mejor_calidad = contraste
                mejor_cliplimit = clip_limit
                mejor_tilegrid = tile_size
                mejor_imagen = imagen_clahe_reescalada

    return mejor_cliplimit, mejor_tilegrid, mejor_imagen

def reescalar_imagen(imagen):
    """
    Reescala las intensidades de una imagen al rango [0, 255].
    """
    return cv2.normalize(imagen, None, alpha=0, beta=255, norm_type=cv2.NORM_MINMAX)

# Aplicar la función a una imagen
for i in range(len(imagenes)):
    clip_limit_optimo, tile_size_optimo, imagen_clahe_optima = optimizar_clahe(imagenes[i])

    plt.figure(figsize=(14, 12))

    plt.subplot(2, 1, 1)
    plt.title("Original (RGB)")
    plt.imshow(cv2.cvtColor(imagenes[i], cv2.COLOR_BGR2RGB))
    plt.axis('off')

    print(f"Mejor clipLimit: {clip_limit_optimo}, Mejor tileGridSize: {tile_size_optimo}")

    # Mostrar la imagen optimizada
    plt.subplot(2, 1, 2)
    plt.title("Imagen CLAHE Optimizada")
    plt.imshow(cv2.cvtColor(imagen_clahe_optima, cv2.COLOR_BGR2RGB))
    plt.axis('off')

    plt.tight_layout()
    plt.show()

## Clahe mezclando la imagen procesada con la original

In [ ]:
def optimizar_clahe(imagen, rango_cliplimit=(2.0, 10.0), tamanos_tiles=[(4, 4), (8, 8), (16, 16)]):
    mejor_calidad = -float('inf')  # Inicializamos una métrica de calidad
    mejor_cliplimit = None
    mejor_tilegrid = None
    mejor_imagen = None

    for clip_limit in np.arange(rango_cliplimit[0], rango_cliplimit[1], 0.5):
        for tile_size in tamanos_tiles:
            # Crear el objeto CLAHE con los parámetros actuales
            clahe = cv2.createCLAHE(clipLimit=clip_limit, tileGridSize=tile_size)
            canales_rgb = cv2.split(imagen)
            canales_ecualizados = [clahe.apply(canal) for canal in canales_rgb]
            imagen_clahe = cv2.merge(canales_ecualizados)
            imagen_clahe_reescalada = reescalar_imagen(imagen_clahe)
            imagen_clahe_mezclada = mezclar_imagenes(imagen, imagen_clahe_reescalada, alpha=0.7)

            # Evaluar la calidad del resultado (puedes personalizar esta métrica)
            histograma = cv2.calcHist([cv2.cvtColor(imagen_clahe_mezclada, cv2.COLOR_BGR2GRAY)], [0], None, [256], [0, 256])
            contraste = np.std(histograma)  # Ejemplo: usar la desviación estándar como métrica de contraste

            if contraste > mejor_calidad:  # Si encontramos una mejor calidad
                mejor_calidad = contraste
                mejor_cliplimit = clip_limit
                mejor_tilegrid = tile_size
                mejor_imagen = imagen_clahe_mezclada

    return mejor_cliplimit, mejor_tilegrid, mejor_imagen

def mezclar_imagenes(original, procesada, alpha=0.7):
    """
    Mezcla la imagen original con la procesada en una proporción dada.
    """
    return cv2.addWeighted(original, alpha, procesada, 1 - alpha, 0)

# Aplicar la función a una imagen
for i in range(len(imagenes)):
    clip_limit_optimo, tile_size_optimo, imagen_clahe_optima = optimizar_clahe(imagenes[i])

    plt.figure(figsize=(14, 12))

    plt.subplot(2, 1, 1)
    plt.title("Original (RGB)")
    plt.imshow(cv2.cvtColor(imagenes[i], cv2.COLOR_BGR2RGB))
    plt.axis('off')

    print(f"Mejor clipLimit: {clip_limit_optimo}, Mejor tileGridSize: {tile_size_optimo}")

    # Mostrar la imagen optimizada
    plt.subplot(2, 1, 2)
    plt.title("Imagen CLAHE Optimizada")
    plt.imshow(cv2.cvtColor(imagen_clahe_optima, cv2.COLOR_BGR2RGB))
    plt.axis('off')

    plt.tight_layout()
    plt.show()

## Clahe con gamma

In [ ]:
def optimizar_clahe(imagen, rango_cliplimit=(2.0, 10.0), tamanos_tiles=[(4, 4), (8, 8), (16, 16)]):
    mejor_calidad = -float('inf')  # Inicializamos una métrica de calidad
    mejor_cliplimit = None
    mejor_tilegrid = None
    mejor_imagen = None

    for clip_limit in np.arange(rango_cliplimit[0], rango_cliplimit[1], 0.5):
        for tile_size in tamanos_tiles:
            # Crear el objeto CLAHE con los parámetros actuales
            clahe = cv2.createCLAHE(clipLimit=clip_limit, tileGridSize=tile_size)
            canales_rgb = cv2.split(imagen)
            canales_ecualizados = [clahe.apply(canal) for canal in canales_rgb]
            imagen_clahe = cv2.merge(canales_ecualizados)
            imagen_clahe_gamma = ajustar_gamma(imagen_clahe, gamma=1.2)


            # Evaluar la calidad del resultado (puedes personalizar esta métrica)
            histograma = cv2.calcHist([cv2.cvtColor(imagen_clahe_gamma, cv2.COLOR_BGR2GRAY)], [0], None, [256], [0, 256])
            contraste = np.std(histograma)  # Ejemplo: usar la desviación estándar como métrica de contraste

            if contraste > mejor_calidad:  # Si encontramos una mejor calidad
                mejor_calidad = contraste
                mejor_cliplimit = clip_limit
                mejor_tilegrid = tile_size
                mejor_imagen = imagen_clahe_gamma

    return mejor_cliplimit, mejor_tilegrid, mejor_imagen

def ajustar_gamma(imagen, gamma=1.2):
    """
    Ajusta el brillo de una imagen usando corrección gamma.
    """
    inv_gamma = 1.0 / gamma
    tabla = np.array([(i / 255.0) ** inv_gamma * 255 for i in np.arange(0, 256)]).astype("uint8")
    return cv2.LUT(imagen, tabla)

# Aplicar la función a una imagen
for i in range(len(imagenes)):
    clip_limit_optimo, tile_size_optimo, imagen_clahe_optima = optimizar_clahe(imagenes[i])

    plt.figure(figsize=(14, 12))

    plt.subplot(2, 1, 1)
    plt.title("Original (RGB)")
    plt.imshow(cv2.cvtColor(imagenes[i], cv2.COLOR_BGR2RGB))
    plt.axis('off')

    print(f"Mejor clipLimit: {clip_limit_optimo}, Mejor tileGridSize: {tile_size_optimo}")

    # Mostrar la imagen optimizada
    plt.subplot(2, 1, 2)
    plt.title("Imagen CLAHE Optimizada")
    plt.imshow(cv2.cvtColor(imagen_clahe_optima, cv2.COLOR_BGR2RGB))
    plt.axis('off')

    plt.tight_layout()
    plt.show()

In [ ]:
for i in range(len(imagenes)):
    # Crear CLAHE y aplicar a cada canal
    clahe = cv2.createCLAHE(clipLimit=5.0, tileGridSize=(16, 16))
    canales_rgb = cv2.split(imagenes[i])
    canales_ecualizados = [clahe.apply(canal) for canal in canales_rgb]
    imagen_clahe = cv2.merge(canales_ecualizados)

    # Reescalar las intensidades
    imagen_clahe_reescalada = reescalar_imagen(imagen_clahe)

    # Ajustar Gamma
    imagen_clahe_gamma = ajustar_gamma(imagen_clahe_reescalada, gamma=1.2)

    # Mezclar con la original
    imagen_final = mezclar_imagenes(imagenes[i], imagen_clahe_gamma, alpha=0.7)

    # Mostrar resultados
    plt.figure(figsize=(14, 12))
    plt.subplot(1, 2, 1)
    plt.title("Original (RGB)")
    plt.imshow(cv2.cvtColor(imagenes[i], cv2.COLOR_BGR2RGB))
    plt.axis('off')

    plt.subplot(1, 2, 2)
    plt.title("CLAHE Mejorada")
    plt.imshow(cv2.cvtColor(imagen_final, cv2.COLOR_BGR2RGB))
    plt.axis('off')

    plt.tight_layout()
    plt.show()


In [ ]:
for i in range(len(imagenes)):
    # Ecualizar la imagen en escala de grises

    # Aplicar ecualización a la imagen RGB canal por canal
    canales_rgb = cv2.split(imagenes[i])
    canales_ecualizados = [cv2.normalize(canal, None, alpha=0, beta=255, norm_type=cv2.NORM_MINMAX) for canal in canales_rgb]
    equalized_image_rgb = cv2.merge(canales_ecualizados)

    # Configurar la figura y tamaño
    plt.figure(figsize=(14, 12))

    # Imagen original en RGB
    plt.subplot(3, 2, 5)
    plt.title("Original (RGB)")
    plt.imshow(cv2.cvtColor(imagenes[i], cv2.COLOR_BGR2RGB))
    plt.axis('off')

    # Imagen ecualizada en RGB
    plt.subplot(3, 2, 6)
    plt.title("Ecualizada (RGB)")
    plt.imshow(cv2.cvtColor(equalized_image_rgb, cv2.COLOR_BGR2RGB))
    plt.axis('off')

    # Ajustar los espacios entre subplots
    plt.tight_layout()
    plt.show()


# Operadores aritméticos

## Resta

In [ ]:
# Generar todas las combinaciones de dos imágenes
combinaciones = list(combinations(range(len(imagenes)), 2))
images1_gray = []
images2_gray = []
# Iterar sobre cada par de combinaciones
for idx1, idx2 in combinaciones:
    print(f"Comparando Imagen {idx1 + 1} con Imagen {idx2 + 1}")

    # Leer las imágenes del par
    image1 = imagenes[idx1]
    image2 = imagenes[idx2]

    # Redimensionar la segunda imagen para que coincida con la primera
    image2_resized = cv2.resize(image2, (image1.shape[1], image1.shape[0]))

    # Realizar la operación de resta en color
    result_color = cv2.absdiff(image1, image2_resized)

    # Convertir a escala de grises
    image1_gray = cv2.cvtColor(image1, cv2.COLOR_BGR2GRAY)
    images1_gray.append(image1_gray)
    image2_resized_gray = cv2.cvtColor(image2_resized, cv2.COLOR_BGR2GRAY)
    images2_gray.append(image2_resized_gray)

    # Realizar la operación de resta en escala de grises
    result_gray = cv2.absdiff(image1_gray, image2_resized_gray)

    # Visualizar los resultados para este par
    plt.figure(figsize=(15, 10))

    # Imágenes en color
    plt.subplot(2, 3, 1)
    plt.title(f"Imagen {idx1 + 1} Color")
    plt.imshow(cv2.cvtColor(image1, cv2.COLOR_BGR2RGB))
    plt.axis("off")

    plt.subplot(2, 3, 2)
    plt.title(f"Imagen {idx2 + 1} Color")
    plt.imshow(cv2.cvtColor(image2_resized, cv2.COLOR_BGR2RGB))
    plt.axis("off")

    plt.subplot(2, 3, 3)
    plt.title("Resultado Resta Color")
    plt.imshow(cv2.cvtColor(result_color, cv2.COLOR_BGR2RGB))
    plt.axis("off")

    # Imágenes en escala de grises
    plt.subplot(2, 3, 4)
    plt.title(f"Imagen {idx1 + 1} Escala de grises")
    plt.imshow(image1_gray, cmap="gray")
    plt.axis("off")

    plt.subplot(2, 3, 5)
    plt.title(f"Imagen {idx2 + 1} Escala de grises")
    plt.imshow(image2_resized_gray, cmap="gray")
    plt.axis("off")

    plt.subplot(2, 3, 6)
    plt.title("Resultado Escala de grises")
    plt.imshow(result_gray, cmap="gray")
    plt.axis("off")

    plt.tight_layout()
    plt.show()

    # Mostrar valores de píxeles (opcional)
    fila, columna = 84, 150
    print(f'Pixel Imagen {idx1 + 1} (Color): {image1[fila, columna]}')
    print(f'Pixel Imagen {idx2 + 1} (Color): {image2_resized[fila, columna]}')
    print(f'Pixel Resultado (Color): {result_color[fila, columna]}')
    print(f'Pixel Imagen {idx1 + 1} (Gris): {image1_gray[fila, columna]}')
    print(f'Pixel Imagen {idx2 + 1} (Gris): {image2_resized_gray[fila, columna]}')
    print(f'Pixel Resultado (Gris): {result_gray[fila, columna]}')


In [ ]:
for img in imagenes:

  # Convertir la imagen a RGB (OpenCV la carga en BGR por defecto)
  img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

  # Definir el valor constante

  # Asegúrate de que la imagen esté en formato adecuado
  # Convertir la imagen a tipo float32 para evitar problemas de desbordamiento
  img_float = img_rgb.astype(np.float32)

  C = 30
  # Operación de Suma
  img_suma = img_float + C
  img_suma = np.clip(img_suma, 0, 255)  # Asegurarse de que los valores no se salgan del rango [0, 255]
  img_suma = img_suma.astype(np.uint8)

  c = 30
  # Operación de Resta
  img_resta = img_float + C
  img_resta = np.clip(img_resta, 0, 255)  # Asegurarse de que los valores no se salgan del rango [0, 255]
  img_resta = img_resta.astype(np.uint8)

  c = 0.1
  # Operación de Multiplicación
  img_multiplicacion = img_float * C
  img_multiplicacion = np.clip(img_multiplicacion, 0, 255)  # Asegurarse de que los valores no se salgan del rango [0, 255]
  img_multiplicacion = img_multiplicacion.astype(np.uint8)

  c = 100
  # Operación de División
  img_division = img_float / C
  img_division = np.clip(img_division, 0, 255)  # Asegurarse de que los valores no se salgan del rango [0, 255]
  img_division = img_division.astype(np.uint8)

  # Mostrar las imágenes
  plt.figure(figsize=(15, 10))

  # Imagen original
  plt.subplot(2, 3, 1)
  plt.imshow(img_rgb)
  plt.title("Imagen Original")
  plt.axis('off')

  # Imagen con Suma
  plt.subplot(2, 3, 2)
  plt.imshow(img_suma)
  plt.title("Suma de Constante")
  plt.axis('off')

  # Imagen con Resta
  plt.subplot(2, 3, 3)
  plt.imshow(img_resta)
  plt.title("Resta de Constante")
  plt.axis('off')

  # Imagen con Multiplicación
  plt.subplot(2, 3, 4)
  plt.imshow(img_multiplicacion)
  plt.title("Multiplicación por Constante")
  plt.axis('off')

  # Imagen con División
  plt.subplot(2, 3, 5)
  plt.imshow(img_division)
  plt.title("División por Constante")
  plt.axis('off')

  plt.tight_layout()
  plt.show()


In [ ]:
for imagen in imagenes:
  # Verificar que la imagen se haya cargado correctamente
  if imagen is None:
      print("Error al cargar la imagen")
  else:
      # Sumar un valor constante a la imagen (en cada canal)
      constantes = [50]  # Varias constantes a sumar

      # Convertir la imagen de BGR a RGB para su visualización correcta
      imagen_rgb = cv2.cvtColor(imagen, cv2.COLOR_BGR2RGB)

      # Crear una figura para mostrar las imágenes
      plt.figure(figsize=(20, 7))

      # Mostrar la imagen original en el primer gráfico
      plt.subplot(1, len(constantes) + 1, 1)  # El primer subgráfico
      plt.imshow(imagen_rgb)
      plt.title('Imagen Original')
      plt.axis('off')  # Ocultar los ejes

      # Realizar las sumas y mostrar las imágenes resultantes
      for i, constante in enumerate(constantes):
          # Sumar una constante a la imagen
          imagen_sumada = cv2.add(imagen, (constante, constante, constante, 0))

          # Convertir la imagen sumada de BGR a RGB para su visualización
          imagen_sumada_rgb = cv2.cvtColor(imagen_sumada, cv2.COLOR_BGR2RGB)

          # Mostrar la imagen sumada en los subgráficos
          plt.subplot(1, len(constantes) + 1, i + 2)
          plt.imshow(imagen_sumada_rgb)
          plt.title(f'Suma factor: {constante}')
          plt.axis('off')  # Ocultar los ejes

      # Mostrar todas las imágenes
      plt.show()

In [ ]:
for imagen in imagenes:
  # Verificar que la imagen se haya cargado correctamente
  if imagen is None:
      print("Error al cargar la imagen")
  else:
      # Sumar un valor constante a la imagen (en cada canal)
      constantes = [5]  # Varias constantes a sumar

      # Convertir la imagen de BGR a RGB para su visualización correcta
      imagen_rgb = cv2.cvtColor(imagen, cv2.COLOR_BGR2RGB)

      # Crear una figura para mostrar las imágenes
      plt.figure(figsize=(20, 7))

      # Mostrar la imagen original en el primer gráfico
      plt.subplot(1, len(constantes) + 1, 1)  # El primer subgráfico
      plt.imshow(imagen_rgb)
      plt.title('Imagen Original')
      plt.axis('off')  # Ocultar los ejes

      # Realizar las sumas y mostrar las imágenes resultantes
      for i, constante in enumerate(constantes):
          # Sumar una constante a la imagen
          imagen_sumada = cv2.subtract(imagen, (constante, constante, constante, 0))

          # Convertir la imagen sumada de BGR a RGB para su visualización
          imagen_sumada_rgb = cv2.cvtColor(imagen_sumada, cv2.COLOR_BGR2RGB)

          # Mostrar la imagen sumada en los subgráficos
          plt.subplot(1, len(constantes) + 1, i + 2)
          plt.imshow(imagen_sumada_rgb)
          plt.title(f'Resta factor: {constante}')
          plt.axis('off')  # Ocultar los ejes

      # Mostrar todas las imágenes
      plt.show()

In [ ]:
for imagen in imagenes:

  # Verificar que la imagen se haya cargado correctamente
  if imagen is None:
      print("Error al cargar la imagen")
  else:
      # Sumar un valor constante a la imagen (en cada canal)
      constantes = [9]  # Varias constantes a sumar

      # Convertir la imagen de BGR a RGB para su visualización correcta
      imagen_rgb = cv2.cvtColor(imagen, cv2.COLOR_BGR2RGB)

      # Crear una figura para mostrar las imágenes
      plt.figure(figsize=(20, 7))

      # Mostrar la imagen original en el primer gráfico
      plt.subplot(1, len(constantes) + 1, 1)  # El primer subgráfico
      plt.imshow(imagen_rgb)
      plt.title('Imagen Original')
      plt.axis('off')  # Ocultar los ejes

      # Realizar las sumas y mostrar las imágenes resultantes
      for i, constante in enumerate(constantes):
          # Sumar una constante a la imagen
          imagen_sumada = cv2.multiply(imagen, (constante, constante, constante, 0))

          # Convertir la imagen sumada de BGR a RGB para su visualización
          imagen_sumada_rgb = cv2.cvtColor(imagen_sumada, cv2.COLOR_BGR2RGB)

          # Mostrar la imagen sumada en los subgráficos
          plt.subplot(1, len(constantes) + 1, i + 2)
          plt.imshow(imagen_sumada_rgb)
          plt.title(f'Multiplicación factor: {constante}')
          plt.axis('off')  # Ocultar los ejes

      # Mostrar todas las imágenes
      plt.show()

In [ ]:
for imagen in imagenes:

  # Verificar que la imagen se haya cargado correctamente
  if imagen is None:
      print("Error al cargar la imagen")
  else:
      # Sumar un valor constante a la imagen (en cada canal)
      constantes = [0.1]  # Varias constantes a sumar

      # Convertir la imagen de BGR a RGB para su visualización correcta
      imagen_rgb = cv2.cvtColor(imagen, cv2.COLOR_BGR2RGB)

      # Crear una figura para mostrar las imágenes
      plt.figure(figsize=(20, 7))

      # Mostrar la imagen original en el primer gráfico
      plt.subplot(1, len(constantes) + 1, 1)  # El primer subgráfico
      plt.imshow(imagen_rgb)
      plt.title('Imagen Original')
      plt.axis('off')  # Ocultar los ejes

      # Realizar las sumas y mostrar las imágenes resultantes
      for i, constante in enumerate(constantes):
          # Sumar una constante a la imagen
          imagen_sumada = cv2.divide(imagen, (constante, constante, constante, 0))

          # Convertir la imagen sumada de BGR a RGB para su visualización
          imagen_sumada_rgb = cv2.cvtColor(imagen_sumada, cv2.COLOR_BGR2RGB)

          # Mostrar la imagen sumada en los subgráficos
          plt.subplot(1, len(constantes) + 1, i + 2)
          plt.imshow(imagen_sumada_rgb)
          plt.title(f'Division factor: {constante}')
          plt.axis('off')  # Ocultar los ejes

      # Mostrar todas las imágenes
      plt.show()

In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

# Asumimos que 'imagenes' contiene al menos dos imágenes
image1 = imagenes[0]
image2 = imagenes[0]

# Redimensionar la segunda imagen para que coincida con la primera
image2_resized = cv2.resize(image2, (image1.shape[1], image1.shape[0]))

# Función para aclarar una imagen
def aclarar_imagen(image, alpha=2.5, beta=50):
    # 'alpha' es el factor de contraste (mayor > mayor contraste)
    # 'beta' es el factor de brillo (mayor > más brillo)
    return cv2.convertScaleAbs(image, alpha=alpha, beta=beta)

# Aclarar una de las imágenes (por ejemplo, la imagen 2)
image2_aclarada = aclarar_imagen(image2_resized)

# Realizar la operación de resta en color entre la imagen original y la aclarada
result_color = cv2.absdiff(image1, image2_aclarada)

# Convertir a escala de grises para realizar la operación de resta en grises
image1_gray = cv2.cvtColor(image1, cv2.COLOR_BGR2GRAY)
image2_aclarada_gray = cv2.cvtColor(image2_aclarada, cv2.COLOR_BGR2GRAY)

# Resta en escala de grises
result_gray = cv2.absdiff(image1_gray, image2_aclarada_gray)

# Visualizar los resultados
plt.figure(figsize=(15, 10))

# Imágenes en color
plt.subplot(2, 3, 1)
plt.title(f"Imagen 1 Color")
plt.imshow(cv2.cvtColor(image1, cv2.COLOR_BGR2RGB))
plt.axis("off")

plt.subplot(2, 3, 2)
plt.title(f"Imagen 2 Aclarada")
plt.imshow(cv2.cvtColor(image2_aclarada, cv2.COLOR_BGR2RGB))
plt.axis("off")

plt.subplot(2, 3, 3)
plt.title("Resultado Resta Color")
plt.imshow(cv2.cvtColor(result_color, cv2.COLOR_BGR2RGB))
plt.axis("off")

# Imágenes en escala de grises
plt.subplot(2, 3, 4)
plt.title(f"Imagen 1 Escala de grises")
plt.imshow(image1_gray, cmap="gray")
plt.axis("off")

plt.subplot(2, 3, 5)
plt.title(f"Imagen 2 Aclarada Escala de grises")
plt.imshow(image2_aclarada_gray, cmap="gray")
plt.axis("off")

plt.subplot(2, 3, 6)
plt.title("Resultado Resta Escala de grises")
plt.imshow(result_gray, cmap="gray")
plt.axis("off")

plt.tight_layout()
plt.show()

# Mostrar valores de píxeles en una posición específica
fila, columna = 84, 150
print(f'Pixel Resultado (Color): {result_color[fila, columna]}')
print(f'Pixel Resultado (Gris): {result_gray[fila, columna]}')


In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

# Asumimos que 'imagenes' contiene al menos dos imágenes
image1 = imagenes[0]

# Aplicar un filtro Gaussiano a la imagen
# Puedes ajustar el tamaño del kernel y la desviación estándar
image1_gauss = cv2.GaussianBlur(image1, (5, 5), 1)

# Sumar la imagen original con la imagen suavizada
result = cv2.add(image1, image1_gauss)

# Visualizar los resultados
plt.figure(figsize=(10, 7))

# Imágenes en color
plt.subplot(1, 3, 1)
plt.title("Imagen Original")
plt.imshow(cv2.cvtColor(image1, cv2.COLOR_BGR2RGB))
plt.axis("off")

plt.subplot(1, 3, 2)
plt.title("Imagen Suavizada (Gaussiana)")
plt.imshow(cv2.cvtColor(image1_gauss, cv2.COLOR_BGR2RGB))
plt.axis("off")

plt.subplot(1, 3, 3)
plt.title("Resultado Suma")
plt.imshow(cv2.cvtColor(result, cv2.COLOR_BGR2RGB))
plt.axis("off")

plt.tight_layout()
plt.show()

# Mostrar valores de píxeles en una posición específica (opcional)
fila, columna = 84, 150
print(f'Pixel Resultado (Suma): {result[fila, columna]}')


In [ ]:

import cv2
import numpy as np
import matplotlib.pyplot as plt

# Asumimos que 'imagenes' contiene al menos una imagen
image1 = imagenes[0]

# Aplicar un filtro Gaussiano más fuerte (mayor kernel y mayor sigma)
image1_gauss = cv2.GaussianBlur(image1, (9, 9), 3)  # 9x9 kernel, sigma=3

# Sumar la imagen original con la imagen suavizada
result = cv2.add(image1, image1_gauss)

# Visualizar los resultados
plt.figure(figsize=(10, 7))

# Imágenes en color
plt.subplot(1, 3, 1)
plt.title("Imagen Original")
plt.imshow(cv2.cvtColor(image1, cv2.COLOR_BGR2RGB))
plt.axis("off")

plt.subplot(1, 3, 2)
plt.title("Imagen Suavizada (Gaussiana más fuerte)")
plt.imshow(cv2.cvtColor(image1_gauss, cv2.COLOR_BGR2RGB))
plt.axis("off")

plt.subplot(1, 3, 3)
plt.title("Resultado Suma (Aclarado)")
plt.imshow(cv2.cvtColor(result, cv2.COLOR_BGR2RGB))
plt.axis("off")

plt.tight_layout()
plt.show()

# Mostrar valores de píxeles en una posición específica (opcional)
fila, columna = 84, 150
print(f'Pixel Resultado (Suma): {result[fila, columna]}')


In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

# Cargar la imagen original (supongo que 'imagenes' tiene las imágenes)
image1 = imagenes[0]  # Imagen nocturna

# Aumentar el brillo de la imagen utilizando la escala lineal
alpha = 2.0  # Contraste (aumenta el valor para mejorar el contraste)
beta = 50    # Brillo (aumenta para hacer la imagen más brillante)
bright_image = cv2.convertScaleAbs(image1, alpha=alpha, beta=beta)

# Realizar la operación de resta entre la imagen brillante y la original
result_subtract = cv2.subtract(bright_image, image1)

# Convertir la imagen resultante a un formato que sea visible
result_subtract = cv2.convertScaleAbs(result_subtract)

# Asegurarse de que no haya valores fuera del rango [0, 255]
result_subtract = np.clip(result_subtract, 0, 255)

# Visualizar los resultados
plt.figure(figsize=(10, 7))

# Imágenes en color
plt.subplot(1, 3, 1)
plt.title("Imagen Original (Noche)")
plt.imshow(cv2.cvtColor(image1, cv2.COLOR_BGR2RGB))
plt.axis("off")

plt.subplot(1, 3, 2)
plt.title("Imagen Brillante (Contraste Aumentado)")
plt.imshow(cv2.cvtColor(bright_image, cv2.COLOR_BGR2RGB))
plt.axis("off")

plt.subplot(1, 3, 3)
plt.title("Resultado Resta (Aclarado)")
plt.imshow(cv2.cvtColor(result_subtract, cv2.COLOR_BGR2RGB))
plt.axis("off")

plt.tight_layout()
plt.show()

# Mostrar valores de píxeles (opcional)
fila, columna = 84, 150
print(f'Pixel Resultado (Resta): {result_subtract[fila, columna]}')


Las zonas blancas de la imagen original, se ven quemadas

In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

# Cargar la imagen original (suponiendo que 'imagenes' tiene las imágenes)
image1 = imagenes[0]  # Imagen nocturna

# Aumentar el brillo de la imagen utilizando la escala lineal (sin saturar)
alpha = 1.5  # Contraste (ajustar según necesidad)
beta = 50    # Brillo (ajustar para hacer la imagen más brillante)
bright_image = cv2.convertScaleAbs(image1, alpha=alpha, beta=beta)

# Realizar la operación de resta entre la imagen brillante y la original
result_subtract = cv2.subtract(bright_image, image1)

# Asegurarse de que la imagen resultante no esté fuera de rango [0, 255]
result_subtract = np.clip(result_subtract, 0, 255)

# Visualizar los resultados
plt.figure(figsize=(10, 7))

# Imágenes en color
plt.subplot(1, 3, 1)
plt.title("Imagen Original (Noche)")
plt.imshow(cv2.cvtColor(image1, cv2.COLOR_BGR2RGB))
plt.axis("off")

plt.subplot(1, 3, 2)
plt.title("Imagen Brillante (Ajustada)")
plt.imshow(cv2.cvtColor(bright_image, cv2.COLOR_BGR2RGB))
plt.axis("off")

plt.subplot(1, 3, 3)
plt.title("Resultado de la Resta (Aclarado)")
plt.imshow(cv2.cvtColor(result_subtract, cv2.COLOR_BGR2RGB))
plt.axis("off")

plt.tight_layout()
plt.show()

# Mostrar valores de píxeles (opcional)
fila, columna = 84, 150
print(f'Pixel Resultado Resta: {result_subtract[fila, columna]}')


In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

# Cargar la imagen original (suponiendo que 'imagenes' tiene las imágenes)
image1 = imagenes[0]  # Imagen nocturna

# Definir el factor de multiplicación (ajustar este valor según la imagen)
alpha = 5.0  # Aumento de brillo y contraste

# Multiplicar la imagen por un valor 'alpha' para aumentar el brillo/contraste
image_multiplied = cv2.convertScaleAbs(image1, alpha=alpha, beta=0)

# Visualizar los resultados
plt.figure(figsize=(10, 7))

# Imágenes en color
plt.subplot(1, 2, 1)
plt.title("Imagen Original")
plt.imshow(cv2.cvtColor(image1, cv2.COLOR_BGR2RGB))
plt.axis("off")

plt.subplot(1, 2, 2)
plt.title(f"Imagen Multiplicada (Factor {alpha})")
plt.imshow(cv2.cvtColor(image_multiplied, cv2.COLOR_BGR2RGB))
plt.axis("off")

plt.tight_layout()
plt.show()

# Mostrar valores de píxeles (opcional)
fila, columna = 84, 150
print(f'Pixel Resultado (Multiplicación): {image_multiplied[fila, columna]}')


In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

# Cargar la imagen original (suponiendo que 'imagenes' tiene las imágenes)
image1 = imagenes[0]  # Imagen nocturna

# Definir el factor de división (ajustar este valor según la imagen)
factor_division = 0.35  # Factor de división

# Dividir la imagen por el valor del factor para aclarar las zonas oscuras
image_divided = np.clip(image1 / factor_division, 0, 255).astype(np.uint8)

# Visualizar los resultados
plt.figure(figsize=(10, 7))

# Imágenes en color
plt.subplot(1, 2, 1)
plt.title("Imagen Original")
plt.imshow(cv2.cvtColor(image1, cv2.COLOR_BGR2RGB))
plt.axis("off")

plt.subplot(1, 2, 2)
plt.title(f"Imagen Dividida (Factor {factor_division})")
plt.imshow(cv2.cvtColor(image_divided, cv2.COLOR_BGR2RGB))
plt.axis("off")

plt.tight_layout()
plt.show()

# Mostrar valores de píxeles (opcional)
fila, columna = 84, 150
print(f'Pixel Resultado (División): {image_divided[fila, columna]}')


## Negativo + CLAHE

In [ ]:
# Aplicar CLAHE en RGB
def clahe_rgb(img):
    lab = cv2.cvtColor(img, cv2.COLOR_RGB2LAB)
    l, a, b = cv2.split(lab)
    clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8, 8))
    l_clahe = clahe.apply(l)
    lab_clahe = cv2.merge((l_clahe, a, b))
    return cv2.cvtColor(lab_clahe, cv2.COLOR_LAB2RGB)

# Corrección gamma
def gamma_correction_rgb(img, gamma=1.5):
    inv_gamma = 1.0 / gamma
    table = np.array([(i / 255.0) ** inv_gamma * 255 for i in np.arange(0, 256)]).astype("uint8")
    return cv2.LUT(img, table)

def log_transform_rgb(img):
    # Normalizar la imagen al rango [0, 1]
    img_normalized = img / 255.0
    # Aplicar la transformación logarítmica
    c = 1 / np.log(1 + np.max(img_normalized))
    log_image = c * np.log(1 + img_normalized)
    # Escalar de regreso al rango [0, 255]
    log_image_scaled = np.uint8(255 * log_image)
    return log_image_scaled

# Ecualización global
def global_hist_equalization_rgb(img):
    ycrcb = cv2.cvtColor(img, cv2.COLOR_RGB2YCrCb)
    y, cr, cb = cv2.split(ycrcb)
    y_eq = cv2.equalizeHist(y)
    ycrcb_eq = cv2.merge((y_eq, cr, cb))
    return cv2.cvtColor(ycrcb_eq, cv2.COLOR_YCrCb2RGB)

# Combinación de algoritmos
for i, img in enumerate(imagenes):
    # Convertir a RGB
    img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

    # Aplicar los algoritmos
    img_clahe = clahe_rgb(img_rgb)
    img_gamma = gamma_correction_rgb(img_clahe, gamma=1.5)  # Aplicar gamma tras CLAHE
    img_log = log_transform_rgb(img_gamma)  # Aplicar logarítmica después de gamma
    img_eq_global = global_hist_equalization_rgb(img_log)  # Aplicar ecualización global final

    # Mostrar los resultados
    fig, axes = plt.subplots(1, 5, figsize=(20, 10))
    titles = ["Original", "CLAHE", "CLAHE + Gamma", "CLAHE + Gamma + Log", "CLAHE + Gamma + Log + Eq Global"]
    images = [img_rgb, img_clahe, img_gamma, img_log, img_eq_global]

    for ax, title, transformed_img in zip(axes, titles, images):
        ax.imshow(transformed_img)
        ax.set_title(title)
        ax.axis('off')

    plt.tight_layout()
    plt.show()
